Imports

In [1]:
import pandas as pd
import numpy as np
import random
import scipy
import json
import nltk
import math
import re
import threading
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize.regexp import (WordPunctTokenizer,wordpunct_tokenize)

Links for the datasets

In [2]:
url_links = 'https://zrekoj.github.io/hybrid-recommender-system/dataset/links.csv'
url_movies = 'https://zrekoj.github.io/hybrid-recommender-system/dataset/movies.csv'
url_ratings = 'https://zrekoj.github.io/hybrid-recommender-system/dataset/ratings.csv'
url_tags = 'https://zrekoj.github.io/hybrid-recommender-system/dataset/tags.csv'
url_genres = 'https://zrekoj.github.io/hybrid-recommender-system/dataset/user_genre.json'

Datasets

In [3]:
ds_links = pd.read_csv(url_links, dtype = str)
ds_movies = pd.read_csv(url_movies, dtype = str)
ds_ratings = pd.read_csv(url_ratings)
ds_tags = pd.read_csv(url_tags, dtype = str)
ds_user_genres = pd.read_json(url_genres)

In [4]:
list_movies_id = list(ds_movies['movieId'].unique())
list_title_id = ds_movies['title'].tolist()
list_users_id = list(ds_ratings['userId'].unique())
list_movies_imdbid = ds_links['imdbId'].tolist()

Input ratings

In [6]:
movie_ids=[]
ratings=[]

for i in range(10):
  index=random.randint(0,len(list_movies_id)-1)
  while list_movies_id[index] in movie_ids:
    index=random.randint(0,len(list_movies_id)-1)
  rating = float(input('Rate the movie '+str(list_title_id[index])+' from 0.5 to 5.0 or say 0.0 if you haven\'t seen it :'))
  movie_ids.append(list_movies_id[index])
  ratings.append(rating)
print(movie_ids)
print(ratings)


Rate the movie 13th (2016) from 0.5 to 5.0 or say 0.0 if you haven't seen it :9


KeyboardInterrupt: 

Collaborative filtering

SVD

In [5]:
def getGenresByUser(user_id, ds=ds_user_genres):
  return ds[user_id]['like'], ds[user_id]['dislike']

print(getGenresByUser(1))

(['Film-Noir', 'Animation', 'Musical', 'Children', 'Drama', 'War'], ['Comedy', 'Sci-Fi', 'Mystery', 'Thriller', 'Horror'])


In [6]:
#hyperparams

test_ratio = 0.2 #fraction of data to be used as test set.
no_of_features = [8,10,12,14,17,20] # to test the performance over a different number of features

In [7]:
ds_ratings['userId'] = ds_ratings['userId'].astype('str')
ds_ratings['movieId'] = ds_ratings['movieId'].astype('str')

users = ds_ratings['userId'].unique() #list of all users
movies = ds_ratings['movieId'].unique() #list of all movies

test = pd.DataFrame(columns=ds_ratings.columns)
train = pd.DataFrame(columns=ds_ratings.columns)

for u in users:
  temp = ds_ratings[ds_ratings['userId'] == u]
  n = len(temp)
  test_size = int(test_ratio*n)

  temp = temp.sort_values('timestamp').reset_index()
  temp.drop('index', axis=1, inplace=True)

  dummy_test = temp.loc[n-1-test_size :]
  dummy_train = temp.loc[: n-2-test_size]
    
  test = pd.concat([test, dummy_test])
  train = pd.concat([train, dummy_train])

In [8]:
from scipy.linalg import sqrtm

def create_user_item_matrix(data, formatizer = {'user':0, 'item': 1, 'value': 2}):
    itemField = formatizer['item']
    userField = formatizer['user']
    valueField = formatizer['value']

    
    userList = data.iloc[:,userField].tolist()
    itemList = data.iloc[:,itemField].tolist()
    valueList = data.iloc[:,valueField].tolist()

    users = list(set(data.iloc[:,userField]))
    items = list(set(data.iloc[:,itemField]))

    users_index = {users[i]: i for i in range(len(users))}

    pd_dict = {item: [np.nan for i in range(len(users))] for item in items}

    for i in range(0,len(data)):
      item = itemList[i]
      user = userList[i]
      value = valueList[i]

      pd_dict[item][users_index[user]] = value
    
    X = pd.DataFrame(pd_dict)
    X.index = users

    itemcols = list(X.columns)
    items_index = {itemcols[i]: i for i in range(len(itemcols))}

    # users_index gives us a mapping of user_id to index of user
    # items_index provides the same for items

    return X, users_index, items_index

In [9]:
def svd(train, k):
    utilMat = np.array(train)

    # the nan or unavailable entries are masked
    mask = np.isnan(utilMat)
    masked_arr = np.ma.masked_array(utilMat, mask)

    item_means = np.mean(masked_arr, axis=0)

    # nan entries will replaced by the average rating for each item
    utilMat = masked_arr.filled(item_means)
    x = np.tile(item_means, (utilMat.shape[0],1))

    # we remove the per item average from all entries.
    # the above mentioned nan entries will be essentially zero now
    utilMat = utilMat - x

    # The magic happens here. U and V are user and item features
    U, s, V=np.linalg.svd(utilMat, full_matrices=False)
    s=np.diag(s)

    # we take only the k most significant features
    s=s[0:k,0:k]
    U=U[:,0:k]
    V=V[0:k,:]

    s_root=sqrtm(s)
    Usk=np.dot(U,s_root)
    skV=np.dot(s_root,V)
    UsV = np.dot(Usk, skV)
    UsV = UsV + x
    return UsV

In [10]:
def rmse(true, pred):
    x = true - pred
    return sum([xi*xi for xi in x])/len(x)

In [11]:
uiMat, users_index, items_index = create_user_item_matrix(train)

In [12]:
#svd evaluation
for f in no_of_features:
  svdout = svd(uiMat, k=f)
  pred = [] #to store the predicted ratings
  for _,row in test.iterrows():
    user = row['userId']
    item = row['movieId']
    u_index = users_index[user]
    
    if item in items_index:
      i_index = items_index[item]
      pred_rating = svdout[u_index, i_index]
    else:
      pred_rating = np.mean(svdout[u_index, :])
    pred.append(pred_rating)
  print(rmse(test['rating'], pred))

1.0037878092372214
1.0036410576245889
1.0047280968304317
1.005385440971549
1.0046710025637098
1.0056567229647206


In [13]:
def filter(u_id, user_genres = ds_user_genres, item_genres = ds_movies["genres"]):
  u_likes = user_genres[u_id]["like"]
  u_detest = user_genres[u_id]["dislike"]

  useritemMat, users_index, items_index = create_user_item_matrix(ds_ratings)
  svdout = svd(useritemMat, k=12)
  user_pos = users_index[str(u_id)]
  result = []
  for i in range(len(svdout[user_pos])):

    if str(i) in items_index:
      i_id = items_index[str(i)]
      i_genre = item_genres[i_id].split(sep="|")

      checkDislike =  any(item in i_genre for item in u_detest)
      if checkDislike is False:
        new_rate = svdout[user_pos][i]

        checkLike =  any(item in i_genre for item in u_likes)
        if checkLike is True:
          new_rate = new_rate + 2
        result.append([ds_movies['movieId'][i_id], ds_movies["title"][i_id], new_rate])
 
  return sorted(result, key=lambda item: item[2], reverse=True)[:20]



In [14]:
check =  any(item in ["Horror", "Comedy"] for item in ["Comedy", "Sci-Fi", "Mystery", "Thriller"])
if check is True:
    print("pass") 

pass


In [16]:
filter(1)

[['25937', 'Easter Parade (1948)', 7.0],
 ['385', 'Man of No Importance, A (1994)', 7.0],
 ['6832', 'Regarding Henry (1991)', 7.0],
 ['6216', 'Nowhere in Africa (Nirgendwo in Afrika) (2001)', 7.0],
 ['64499', 'Che: Part One (2008)', 7.0],
 ['1507', 'Paradise Road (1997)', 7.0],
 ['1415', 'Thieves (Voleurs, Les) (1996)', 7.0],
 ['1401', 'Ghosts of Mississippi (1996)', 7.0],
 ['3000', 'Princess Mononoke (Mononoke-hime) (1997)', 7.0],
 ['5471', 'Perfect (1985)', 7.0],
 ['4896',
  "Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)",
  7.0],
 ['55274', 'Elizabeth: The Golden Age (2007)', 7.0],
 ['7243', "Intolerance: Love's Struggle Throughout the Ages (1916)", 7.0],
 ['851', 'Basquiat (1996)', 7.0],
 ['25753', 'Greed (1924)', 7.0],
 ['918', 'Meet Me in St. Louis (1944)', 7.0],
 ['114795', 'Dracula Untold (2014)', 7.0],
 ['54997', '3:10 to Yuma (2007)', 7.0],
 ['157122', 'The Man Who Knew Infinity (2016)', 7.0],
 ['6777', 'Judgment at Nuremberg (

**Content-based**

In [15]:
def cosine_similarity(u, v):
    return np.dot(u, np.transpose(v)) / np.linalg.norm(u) * np.linalg.norm(v)

In [16]:
def readPlots(movies):
  plots = []
  for movie in movies:
    url = 'https://zrekoj.github.io/hybrid-recommender-system/movies/'+ str(movie)+'.json'
    archivo = pd.read_json(url)
    plots.append([archivo['Plot'].tolist()[0]])
  return plots


Build Frecuency Dictionary

In [17]:
def preprocess_text(doc):
 stopset = set(stopwords.words('english'))
 stemmer = SnowballStemmer('english',ignore_stopwords=True)
 tokens = wordpunct_tokenize(doc)
 clean = [token.lower() for token in tokens if token.lower() not in stopset and len(token) > 2]
 stemmed_text = [stemmer.stem(word) for word in clean]
 return stemmed_text
 
 
def create_freq_dict(movieIds, preprocessed_texts):
 i=0
 print("Creating frequency dictionary")
 freqDict_list=[]
 all_tokens=set()
 #for j in range(0, len(preprocessed_texts)-1):
   #for token in preprocessed_texts[j]:
     #all_tokens.add(token)
 for tokens in preprocessed_texts:
  freq_dict={}
  
  for token in tokens:
    if token in freq_dict:
     freq_dict[token]+=1
    else:
     freq_dict[token]=1
  #for token in all_tokens:
    #if token not in freq_dict:
       #freq_dict[token]=0

  temp={'term_id':i,'movie_id':movieIds[i],'freq_dict':freq_dict,'doc_length':len(tokens)}
  i+=1
  freqDict_list.append(temp)
 return freqDict_list

Calculate TF-IDF

Content-Based application

In [18]:
def computeTF(freqDict_list):
 TF_scores=[]
 print("Calculating TF")
 for tempDict in freqDict_list:
  id=tempDict['term_id']
  name=tempDict['movie_id']
  for k in tempDict['freq_dict']:
   temp={'term_id':id, 'movie_id':name,'TF_score':tempDict['freq_dict'][k]/tempDict['doc_length'],'key':k}
   TF_scores.append(temp)
 return TF_scores
 
def computeIDF(freqDict_list):
  IDF_scores=[]
  print("Calculating IDF")
  counter=-1
  for dict in freqDict_list:
   counter+=1
   for k in dict['freq_dict'].keys():
    count=sum([k in tempDict['freq_dict'] for tempDict in freqDict_list])
    temp= {'term_id': counter, 'IDF_score':math.log(dict['doc_length'])/count,'key':k}
    IDF_scores.append(temp)
  return IDF_scores
 
def computeTFIDF(TF_scores, IDF_scores):
 TF_IDF_scores=[]
 print("Calculating TF-IDF")
 
 for j in IDF_scores:
  for i in TF_scores:
   if j['key']==i['key'] and j['term_id']==i['term_id']:
    temp={'term_id':j['term_id'], 'movie_id':i['movie_id'], 'TFIDF_score': j['IDF_score']*i['TF_score'], 'key':i['key']}
    TF_IDF_scores.append(temp)
 return TF_IDF_scores

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Zihao\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [31]:
#zihao

import requests 

user_id = 1
hybrid = False


movie_similarities=[]

#Get imbdIds of user movies
user_movies = list(
    map(
        lambda movie_id : str(ds_links.loc[ds_links['movieId'] == str(movie_id), 'imdbId'].values[0]), 
        ds_ratings.loc[ds_ratings['userId'] == str(user_id), 'movieId'].tolist()
    )
)
#collaborative_filtering_result = filter(user_id)
movies=[]

#If we are using content-based as part of the hybrid filter, it uses the results of the collaborative filtering
#Otherwise it uses all movies
if(hybrid):
    movies = list(
        map(
            lambda result : ds_links.loc[ds_links['movieId'] == str(result[0]), 'imdbId'].values[0], 
            collaborative_filtering_result
        )
    )
else:
    movies=[str(movie_id) for movie_id in list_movies_imdbid if movie_id not in user_movies]
    
freqDict_list = pd.read_json('https://zrekoj.github.io/hybrid-recommender-system/user_frequency/'+ str(user_id)+'.json')

print(freqDict_list)


{'0114709': {'realiz': 1, 'worri': 1, 'becom': 1, 'life': 2, 'know': 1, 'gave': 1, 'figur': 1, 'favorit': 1, 'doll': 1, 'move': 2, 'way': 1, 'pass': 1, 'ruthless': 1, 'find': 1, 'good': 1, 'killer': 1, 'without': 1, 'famili': 1, 'littl': 1, 'come': 1, 'sid': 1, 'parti': 1, 'lost': 1, 'action': 1, 'phillip': 1, 'howev': 1, 'back': 1, 'woodi': 6, 'rid': 1, 'mother': 1, 'consum': 1, 'name': 2, 'room': 1, 'tri': 1, 'love': 1, 'lightyear': 1, 'especi': 1, 'get': 2, 'toy': 6, 'play': 1, 'must': 2, 'birthday': 1, 'buzz': 3, 'quick': 1, 'jealousi': 1, 'boy': 1, 'known': 1, 'andi': 7, 'new': 1, 'believ': 2}, '0113228': {'year': 1, 'nobodi': 1, 'john': 2, 'snare': 1, 'shop': 1, 'counti': 1, 'drink': 1, 'life': 2, 'someth': 1, 'seem': 1, 'thing': 1, 'ago': 1, 'smile': 1, 'women': 1, 'bait': 1, 'hunter': 1, 'chang': 2, 'might': 2, 'fish': 1, 'claim': 1, 'left': 1, 'much': 1, 'peopl': 1, 'fishermen': 1, 'town': 1, 'smoke': 1, 'wabasha': 1, 'max': 3, 'actual': 1, 'catch': 1, 'gigant': 1, 'month': 1,

In [17]:
#zihao

import math

# invulnerancia de pares
def create_freq_dict(movieIds, preprocessed_texts):
    i=0
    print("Creating frequency dictionary")
    freqDict_list=[]

    for tokens in preprocessed_texts:
        freq_dict={}
  
        for token in tokens:
            if token in freq_dict:
                freq_dict[token]+=1
            else:
                freq_dict[token]=1

        temp={'term_id':i,'movie_id':movieIds[i],'freq_dict':freq_dict,'doc_length':len(tokens)}
        i+=1
        freqDict_list.append(temp)
    return freqDict_list

#zihao
user_movies = ['0114709', '0113228', '0113277', '0114369', '0114814', '0116367', '0115734', '0112573', '0114287', '0109370', '0112851', '0112508', '0109445', '0109686', '0109707', '0076759', '0110912', '0111282', '0114694', '0109444', '0109830', '0110213', '0110475', '0109303', '0106677', '0106977', '0107290', '0107614', '0108052', '0108174', '0108333', '0108358', '0099348', '0096895', '0102926', '0032910', '0116282', '0117060', '0116683', '0117705', '0117500', '0117998', '0116629', '0117628', '0032138', '0033467', '0029843', '0039420', '0031679', '0072951', '0063819', '0038166', '0057546', '0033563', '0076538', '0066817', '0043274', '0117887', '0116409', '0117802', '0067992', '0079470', '0105236', '0091763', '0103772', '0083866', '0096754', '0071853', '0080684', '0093779', '0082971', '0066921', '0078788', '0086190', '0099685', '0078748', '0054215', '0080455', '0093058', '0097499', '0045061', '0088247', '0023969', '0081505', '0107048', '0088763', '0091203', '0072431', '0032455', '0097576', '0084503', '0013442', '0103776', '0105435', '0104691', '0119640', '0118702', '0119229', '0118655', '0118880', '0119094', '0119654', '0082198', '0119488', '0119468', '0119174', '0119345', '0120201', '0118715', '0120888', '0120491', '0120769', '0120890', '0122718', '0020629', '0075148', '0091369', '0093409', '0089218', '0099088', '0034492', '0120815', '0088814', '0091059', '0091149', '0097523', '0120768', '0061852', '0076618', '0089908', '0102803', '0053285', '0038969', '0084827', '0087469', '0077869', '0070016', '0084649', '0090633', '0089469', '0088323', '0094721', '0096446', '0105629', '0104257', '0120812', '0099487', '0120586', '0130018', '0120660', '0024216', '0124198', '0155975', '0128445', '0088011', '0090357', '0120863', '0091225', '0072271', '0090555', '0092086', '0138987', '0151804', '0074812', '0063442', '0120735', '0133093', '0139239', '0133189', '0099422', '0120616', '0120915', '0078346', '0081573', '0021814', '0021884', '0034398', '0073629', '0130827', '0158983', '0087332', '0129167', '0094737', '0120657', '0169547', '0082348', '0031397', '0100802', '0061578', '0058150', '0057076', '0055928', '0137523', '0093870', '0096438', '0070328', '0059800', '0120601', '0094012', '0070608', '0120655', '0151137', '0056197', '0120689', '0064276', '0134119', '0104187', '0105417', '0105793', '0134084', '0102138', '0103060', '0108308', '0087985', '0093105', '0107050', '0089457', '0102057', '0093773', '0172495', '0215129', '0071807', '0071230', '0079501', '0082694', '0067741', '0090728', '0162650', '0120903', '0103241', '0092106', '0066026']
processedText = [['hunter', 'prey', 'neil', 'profession', 'crimin', 'crew', 'hunt', 'score', 'big', 'money', 'target', 'bank', 'vault', 'armor', 'car', 'turn', 'hunt', 'vincent', 'hanna', 'team', 'cop', 'robberi', 'homicid', 'polic', 'divis', 'botch', 'job', 'put', 'hanna', 'onto', 'trail', 'regroup', 'tri', 'put', 'togeth', 'one', 'last', 'big', 'retir', 'score', 'neil', 'vincent', 'similar', 'mani', 'way', 'includ', 'troubl', 'person', 'live', 'crucial', 'moment', 'life', 'neil', 'disobey', 'dictum', 'taught', 'long', 'ago', 'crimin', 'mentor', '--', 'never', 'anyth', 'life', 'walk', 'thirti', 'second', 'flat', 'spot', 'heat', 'come', 'around', 'corner', '--', 'fall', 'love', 'thus', 'stage', 'set', 'suspens', 'end', '....'], ['inventor', 'gepetto', 'creat', 'wooden', 'marionett', 'call', 'pinocchio', 'wish', 'pinocchio', 'real', 'boy', 'unexpect', 'grant', 'fairi', 'fairi', 'assign', 'jimini', 'cricket', 'act', 'pinocchio', 'conscienc', 'keep', 'troubl', 'jimini', 'success', 'endeavor', 'film', 'spent', 'pinocchio', 'deep', 'troubl'], ['eccentr', 'habit', 'baffl', 'strang', 'film', 'director', 'edward', 'wood', 'hollywood', 'outcast', 'nevertheless', 'help', 'former', 'famous', 'bela', 'lugosi', 'devot', 'cast', 'crew', 'show', 'busi', 'misfit', 'believ', 'kilter', 'vision', 'filmmak', 'abl', 'bring', 'overs', 'dream', 'cinemat', 'life', 'despit', 'lack', 'critic', 'commerci', 'success', 'friend', 'manag', 'creat', 'odd', 'endear', 'seri', 'extrem', 'low', 'budget', 'film'], ['forrest', 'gump', 'simpl', 'man', 'low', 'good', 'intent', 'run', 'childhood', 'best', 'friend', 'jenni', 'mama', 'teach', 'way', 'life', 'leav', 'choos', 'destini', 'forrest', 'join', 'armi', 'servic', 'vietnam', 'find', 'new', 'friend', 'call', 'dan', 'bubba', 'win', 'medal', 'creat', 'famous', 'shrimp', 'fish', 'fleet', 'inspir', 'peopl', 'jog', 'start', 'ping', 'pong', 'craze', 'creat', 'smiley', 'write', 'bumper', 'sticker', 'song', 'donat', 'peopl', 'meet', 'presid', 'sever', 'time', 'howev', 'irrelev', 'forrest', 'think', 'childhood', 'sweetheart', 'jenni', 'curran', 'mess', 'life', 'although', 'end', 'want', 'prove', 'anyon', 'love', 'anyon'], ['juli', '2nd', 'communic', 'system', 'worldwid', 'sent', 'chao', 'strang', 'atmospher', 'interfer', 'soon', 'learn', 'militari', 'number', 'enorm', 'object', 'collis', 'cours', 'earth', 'first', 'thought', 'meteor', 'later', 'reveal', 'gigant', 'spacecraft', 'pilot', 'mysteri', 'alien', 'speci', 'attempt', 'communic', 'alien', 'nowher', 'david', 'levinson', 'scientist', 'turn', 'cabl', 'technician', 'discov', 'alien', 'go', 'attack', 'major', 'point', 'around', 'globe', 'less', 'day', 'juli', '3rd', 'alien', 'obliter', 'new', 'york', 'los', 'angel', 'washington', 'well', 'pari', 'london', 'houston', 'moscow', 'survivor', 'set', 'convoy', 'toward', 'area', 'strang', 'govern', 'test', 'ground', 'rumor', 'militari', 'captur', 'alien', 'spacecraft', 'survivor', 'devis', 'plan', 'fight', 'back', 'enslav', 'alien', 'juli', '4th', 'becom', 'day', 'human', 'fight', 'freedom', 'juli', '4th', 'independ', 'day', '...'], ['jimmi', 'dove', 'work', 'bomb', 'squad', 'boston', 'alway', 'one', 'tough', 'job', 'one', 'day', 'decid', 'quit', 'forc', 'becom', 'teacher', 'rooki', 'squad', 'day', 'later', 'former', 'partner', 'kill', 'bomb', 'jimmi', 'becom', 'suspici', 'mayb', 'bomb', 'could', 'built', 'former', 'friend', 'begin', 'investig', 'find', 'worst', 'nightmar', 'return', 'past'], ['naiv', 'idealist', 'jefferson', 'smith', 'leader', 'boy', 'ranger', 'appoint', 'lark', 'spineless', 'governor', 'state', 'reunit', 'state', 'senior', 'senat', 'presidenti', 'hope', 'childhood', 'hero', 'senat', 'joseph', 'pain', 'washington', 'howev', 'smith', 'discov', 'mani', 'shortcom', 'polit', 'process', 'earnest', 'goal', 'nation', 'boy', 'camp', 'lead', 'conflict', 'state', 'polit', 'boss', 'jim', 'taylor', 'taylor', 'first', 'tri', 'corrupt', 'smith', 'later', 'attempt', 'destroy', 'smith', 'scandal'], ['chris', 'taylor', 'young', 'naiv', 'american', 'give', 'colleg', 'volunt', 'combat', 'vietnam', 'upon', 'arriv', 'quick', 'discov', 'presenc', 'quit', 'nonessenti', 'consid', 'insignific', 'soldier', 'fought', 'long', 'rest', 'felt', 'effect', 'combat', 'chris', 'two', 'non', 'commiss', 'offic', 'ill', 'temper', 'indestruct', 'staff', 'sergeant', 'robert', 'barn', 'pleasant', 'cooper', 'sergeant', 'elia', 'grodin', 'line', 'drawn', 'two', 'ncos', 'number', 'men', 'platoon', 'illeg', 'kill', 'occur', 'villag', 'raid', 'war', 'continu', 'chris', 'draw', 'toward', 'psycholog', 'meltdown', 'struggl', 'surviv', 'soon', 'realiz', 'fight', 'two', 'battl', 'conflict', 'enemi', 'conflict', 'men', 'within', 'platoon'], ['sign', 'contract', 'jack', 'torranc', 'normal', 'writer', 'former', 'teacher', 'agre', 'take', 'care', 'hotel', 'long', 'violent', 'past', 'put', 'everyon', 'hotel', 'nervous', 'situat', 'jack', 'slowli', 'get', 'violent', 'angri', 'life', 'son', 'danni', 'tri', 'use', 'special', 'talent', 'shine', 'inform', 'peopl', 'outsid', 'whatev', 'go', 'hotel'], ['height', 'war', 'vietnam', 'armi', 'captain', 'willard', 'sent', 'colonel', 'luca', 'general', 'carri', 'mission', 'offici', 'exist', 'ever', 'exist', 'mission', 'seek', 'mysteri', 'green', 'beret', 'colonel', 'walter', 'kurtz', 'whose', 'armi', 'cross', 'border', 'cambodia', 'conduct', 'hit', 'run', 'mission', 'viet', 'cong', 'nva', 'armi', 'believ', 'kurtz', 'gone', 'complet', 'insan', 'willard', 'job', 'elimin', 'willard', 'sent', 'nung', 'river', 'navi', 'patrol', 'boat', 'discov', 'target', 'one', 'decor', 'offic', 'armi', 'crew', 'meet', 'surfer', 'type', 'colonel', 'kilgor', 'head', 'armi', 'helicopt', 'cavalri', 'group', 'elimin', 'viet', 'cong', 'outpost', 'provid', 'entri', 'point', 'nung', 'river', 'hair', 'rais', 'encount', 'crew', 'kill', 'willard', 'lanc', 'chef', 'reach', 'colonel', 'kurtz', 'outpost', 'beyond', 'lung', 'bridg', 'becom', 'prison', 'kurtz', 'willard', 'other', 'abl', 'fulfil', 'mission'], ['distant', 'futur', 'high', 'school', 'kid', 'encourag', 'becom', 'citizen', 'join', 'militari', 'know', 'soon', 'engag', 'full', 'scale', 'war', 'planet', 'alien', 'insect', 'fight', 'ensur', 'safeti', 'human'], ['initech', 'offic', 'insecur', 'peter', 'gibbon', 'hate', 'job', 'abus', 'divis', 'bill', 'lumbergh', 'hire', 'two', 'consult', 'downsiz', 'compani', 'best', 'friend', 'softwar', 'engin', 'michael', 'bolton', 'samir', 'nagheenanajar', 'also', 'hate', 'initech', 'next', 'door', 'neighbor', 'lawrenc', 'girlfriend', 'ann', 'cheat', 'convinc', 'peter', 'visit', 'hypnotherapist', 'swanson', 'peter', 'tell', 'miser', 'life', 'swanson', 'hypnot', 'goe', 'state', 'ecstasi', 'howev', 'swanson', 'die', 'immedi', 'give', 'hypnot', 'suggest', 'peter', 'date', 'waitress', 'joanna', 'chang', 'attitud', 'compani', 'promot', 'consult', 'discov', 'michael', 'samir', 'fire', 'decid', 'plant', 'virus', 'account', 'system', 'embezzl', 'fraction', 'cent', 'financi', 'oper', 'peter', 'account', 'howev', 'michael', 'commit', 'mistak', 'softwar', 'instead', 'decim', 'steal', 'larg', 'amount', 'desper', 'trio', 'tri', 'fix', 'problem', 'prison'], ['philip', 'gaston', 'thief', 'escap', 'dungeon', 'aquila', 'spark', 'manhunt', 'near', 'captur', 'captain', 'navarr', 'befriend', 'navarr', 'hunt', 'bishop', 'men', 'two', 'year', 'ever', 'sinc', 'escap', 'ladi', 'isabeau', 'bishop', 'lust', 'navarr', 'isabeau', 'curs', 'bishop', 'place', 'caus', 'navarr', 'wolf', 'night', 'isabeau', 'hawk', 'day', 'navarr', 'insist', 'philip', 'help', 'enter', 'citi', 'help', 'kill', 'heavili', 'guard', 'bishop'], ['new', 'york', 'polic', 'detect', 'john', 'shaft', 'lead', 'detect', 'sensit', 'case', 'young', 'black', 'man', 'sever', 'beaten', 'man', 'companion', 'tell', 'shaft', 'friend', 'humili', 'one', 'sprout', 'racial', 'slur', 'shaft', 'confront', 'say', 'walter', 'wade', 'son', 'wealthi', 'man', 'shaft', 'find', 'woman', 'waitress', 'bar', 'wade', 'guy', 'attack', 'wade', 'continu', 'hurl', 'racist', 'comment', 'shaft', 'smack', 'shaft', 'later', 'learn', 'action', 'wade', 'grant', 'bail', 'fled', 'two', 'year', 'later', 'wade', 'return', 'shaft', 'arrest', 'hear', 'judg', 'grant', 'bail', 'shaft', 'throw', 'badg', 'judg', 'set', 'get', 'wade', 'find', 'waitress', 'wade', 'meantim', 'ask', 'drug', 'dealer', 'name', 'peopl', 'hernandez', 'find', 'waitress', 'make', 'sure', 'talk'], ['stanley', 'ipkiss', 'jim', 'carrey', 'bank', 'clerk', 'incred', 'nice', 'man', 'unfortun', 'nice', 'good', 'pushov', 'come', 'confront', 'one', 'worst', 'day', 'life', 'find', 'mask', 'depict', 'loki', 'nors', 'night', 'god', 'mischief', 'put', 'becom', 'inner', 'self', 'cartoon', 'romant', 'wild', 'man', 'howev', 'small', 'time', 'crime', 'boss', 'dorian', 'tyrel', 'peter', 'green', 'come', 'across', 'charact', 'dub', 'mask', 'media', 'ipkiss', 'alter', 'ego', 'indirect', 'kill', 'friend', 'crime', 'tyrel', 'want', 'green', 'face', 'goon', 'destroy'], ['former', 'rock', 'star', 'johnni', 'boz', 'brutal', 'kill', 'sex', 'case', 'assign', 'detect', 'nick', 'curran', 'sfpd', 'investig', 'nick', 'meet', 'catherin', 'tramel', 'crime', 'novelist', 'boz', 'girlfriend', 'die', 'catherin', 'prove', 'clever', 'manipul', 'woman', 'though', 'nick', 'less', 'convinc', 'murder', 'boz', 'unabl', 'find', 'evid', 'later', 'nilsen', 'nick', 'rival', 'polic', 'kill', 'nick', 'suspect', 'catherin', 'involv', 'start', 'play', 'danger', 'lust', 'fill', 'mind', 'game', 'catherin', 'nail', 'relationship', 'progress', 'bodi', 'count', 'rise', 'contradict', 'evid', 'forc', 'nick', 'start', 'question', 'suspicion', 'catherin', 'guilt'], ['fbi', 'traine', 'claric', 'starl', 'work', 'hard', 'advanc', 'career', 'includ', 'tri', 'hide', 'put', 'behind', 'west', 'virginia', 'root', 'knew', 'would', 'automat', 'classifi', 'backward', 'white', 'trash', 'graduat', 'aspir', 'work', 'agenc', 'behavior', 'scienc', 'unit', 'leadership', 'jack', 'crawford', 'still', 'traine', 'crawford', 'ask', 'question', 'hannib', 'lecter', 'psychiatrist', 'imprison', 'thus', 'far', 'eight', 'year', 'maximum', 'secur', 'isol', 'serial', 'killer', 'cannib', 'victim', 'claric', 'abl', 'figur', 'assign', 'pick', 'lecter', 'brain', 'help', 'solv', 'anoth', 'serial', 'murder', 'case', 'someon', 'coin', 'media', 'buffalo', 'bill', 'far', 'kill', 'five', 'victim', 'locat', 'eastern', 'young', 'women', 'slight', 'overweight', 'especi', 'around', 'hip', 'drown', 'natur', 'bodi', 'water', 'strip', 'larg', 'swath', 'skin', 'also', 'figur', 'crawford', 'chose', 'woman', 'abl', 'trigger', 'emot', 'respons', 'lecter', 'speak', 'lecter', 'first', 'time', 'realiz', 'everyth', 'psycholog', 'game', 'often', 'read', 'cryptic', 'line', 'provid', 'decid', 'much', 'play', 'along', 'request', 'return', 'talk', 'expos', 'emot', 'case', 'take', 'dire', 'turn', 'sixth', 'victim', 'discov', 'one', 'abl', 'retriev', 'key', 'piec', 'evid', 'lecter', 'forthright', 'mean', 'potenti', 'seventh', 'victim', 'high', 'profil', 'catherin', 'martin', 'daughter', 'senat', 'ruth', 'martin', 'place', 'greater', 'scrutini', 'case', 'search', 'hope', 'still', 'aliv', 'catherin', 'may', 'factor', 'happen', 'frederick', 'chilton', 'warden', 'prison', 'opportunist', 'see', 'higher', 'profil', 'catherin', 'mean', 'higher', 'profil', 'insert', 'success', 'proceed'], ['henri', 'hill', 'might', 'small', 'time', 'gangster', 'may', 'taken', 'part', 'robberi', 'jimmi', 'conway', 'tommi', 'vito', 'two', 'gangster', 'might', 'set', 'sight', 'bit', 'higher', 'two', 'partner', 'could', 'kill', 'everyon', 'els', 'involv', 'robberi', 'slowli', 'start', 'think', 'climb', 'hierarchi', 'mob', 'henri', 'howev', 'might', 'bad', 'affect', 'partner', 'success', 'consid', 'stoop', 'low', 'enough', 'bring', 'downfal', 'jimmi', 'tommi'], ['wayn', 'szalinski', 'averag', 'nutti', 'scientist', 'work', 'top', 'secret', 'machin', 'shrink', 'object', 'unexpect', 'start', 'work', 'amaz', 'forget', 'tell', 'famili', 'care', 'wander', 'lab', '...'], ['uncl', 'remus', 'draw', 'upon', 'tale', 'brer', 'rabbit', 'help', 'littl', 'johnni', 'deal', 'confus', 'parent', 'separ', 'well', 'new', 'life', 'plantat', 'tale', 'briar', 'patch', 'tar', 'babi', 'brer', 'rabbit', 'laugh', 'place'], ['strand', '1955', 'marti', 'mcfli', 'receiv', 'written', 'word', 'friend', 'doctor', 'emmett', 'brown', 'found', 'delorean', 'time', 'machin', 'howev', 'unfortun', 'discoveri', 'prompt', 'marti', 'friend', 'aid', 'use', 'time', 'machin', 'marti', 'travel', 'old', 'west', 'friend', 'run', 'afoul', 'gang', 'thug', 'fallen', 'love', 'local', 'schoolteach', 'use', 'technolog', 'time', 'marti', 'emmett', 'devis', 'one', 'last', 'chanc', 'send', 'two', 'back', 'futur'], ['demon', 'seek', 'creat', 'etern', 'night', 'destroy', 'last', 'unicorn', 'marri', 'fairi', 'princess', 'oppos', 'forest', 'boy', 'jack', 'elven', 'alli', 'magic', 'fantasi', 'two', 'differ', 'version', 'pictur', 'featur', 'soundtrack', 'either', 'tangerin', 'dream', 'jerri', 'goldsmith'], ['lola', 'receiv', 'phone', 'call', 'boyfriend', 'manni', 'lost', '100', '000', 'subway', 'train', 'belong', 'bad', 'guy', 'lola', 'min', 'rais', 'amount', 'meet', 'manni', 'otherwis', 'rob', 'store', 'get', 'money', 'three', 'differ', 'altern', 'may', 'happen', 'depend', 'minor', 'event', 'along', 'lola', 'run'], ['sean', 'archer', 'tough', 'rug', 'fbi', 'agent', 'still', 'griev', 'dead', 'son', 'michael', 'archer', 'believ', 'son', 'killer', 'sworn', 'enemi', 'power', 'crimin', 'castor', 'troy', 'one', 'day', 'archer', 'final', 'corner', 'castor', 'howev', 'fight', 'knock', 'troy', 'cold', 'archer', 'final', 'breath', 'easi', 'captur', 'enemi', 'find', 'troy', 'plant', 'bomb', 'destroy', 'entir', 'citi', 'los', 'angel', 'inhabit', 'unfortun', 'person', 'know', 'locat', 'castor', 'brother', 'pollux', 'refus', 'talk', 'solut', 'special', 'oper', 'doctor', 'cut', 'peopl', 'face', 'place', 'person', 'face', 'onto', 'anoth', 'person', 'archer', 'undergo', 'one', 'surgeri', 'talk', 'pollux', 'howev', 'castor', 'troy', 'somehow', 'regain', 'conscious', 'want', 'reveng', 'archer', 'take', 'face', 'troy', 'ruin', 'archer', 'mission', 'person', 'life', 'well', 'archer', 'must', 'stop', 'troy', 'time', 'person'], ['group', 'friend', 'head', 'las', 'vega', 'bachelor', 'parti', 'thing', 'wrong', 'woman', 'kill', 'soon', 'bodi', 'pile', 'friend', 'find', 'turn', 'one', 'anoth', 'coverup', 'build'], ['larg', 'box', 'arriv', 'donald', 'birthday', 'three', 'gift', 'insid', 'unwrap', 'one', 'time', 'take', 'adventur', 'first', 'movi', 'projector', 'film', 'bird', 'south', 'america', 'donald', 'watch', 'two', 'cartoon', 'one', 'tell', 'penguin', 'long', 'live', 'tropic', 'isl', 'gaucho', 'boy', 'hunt', 'wild', 'ostrich', 'second', 'gift', 'pop', 'book', 'brazil', 'insid', 'jose', 'carioca', 'take', 'donald', 'brazil', 'bahia', 'mix', 'anim', 'live', 'action', 'two', 'cartoon', 'bird', 'sing', 'danc', 'nativ', 'third', 'gift', 'pinata', 'accompani', 'panchito', 'ride', 'magic', 'serap', 'take', 'three', 'amigo', 'sing', 'danc', 'across', 'mexico', 'ole'], ['max', 'fischer', 'precoci', 'year', 'old', 'whose', 'reason', 'live', 'attend', 'rushmor', 'privat', 'school', 'well', 'class', 'king', 'extracurricular', 'activ', 'beekeep', 'societi', 'write', 'produc', 'play', 'littl', 'school', 'life', 'begin', 'chang', 'howev', 'find', 'academ', 'probat', 'stumbl', 'love', 'miss', 'cross', 'pretti', 'teacher', 'elementari', 'school', 'rushmor', 'ad', 'mix', 'friendship', 'herman', 'blume', 'wealthi', 'industrialist', 'father', 'boy', 'attend', 'school', 'also', 'find', 'attract', 'miss', 'cross', 'max', 'fate', 'becom', 'inextric', 'tie', 'odd', 'love', 'triangl', 'set', 'resolv', 'stori', 'film'], ['rout', 'visit', 'grandfath', 'grave', 'appar', 'ritualist', 'desecr', 'five', 'teenag', 'drive', 'past', 'slaughterhous', 'pick', 'quick', 'drop', 'sinist', 'hitch', 'hiker', 'eat', 'delici', 'home', 'cure', 'meat', 'roadsid', 'gas', 'station', 'end', 'old', 'famili', 'home', '...', 'plung', 'never', 'end', 'nightmar', 'meet', 'famili', 'cannib', 'make', 'power', 'tool', 'lack', 'social', 'skill', '...'], ['told', 'three', 'perspect', 'stori', 'bunch', 'young', 'californian', 'tri', 'get', 'cash', 'deal', 'drug', 'score', 'money', 'sex', 'las', 'vega', 'general', 'experi', 'rush', 'life'], ['austin', 'power', 'spi', 'cryonic', 'frozen', 'releas', '1990', 'world', 'differ', 'place', 'power', 'unfortun', 'austin', 'everyon', 'longer', 'sex', 'mad', 'although', 'may', 'differ', 'decad', 'mission', 'still', 'team', 'vanessa', 'kensington', 'stop', 'evil', 'evil', 'also', 'frozen', 'past', 'evil', 'stole', 'nuclear', 'weapon', 'demand', 'payment', 'realis', '100', 'billion', 'dollar', 'austin', 'power', 'stop', 'madman', 'caught', 'evil', 'henchman', 'name', 'like', 'alotta', 'fagina', 'random', 'task', 'time', 'tell'], ['puppet', 'craig', 'schwartz', 'anim', 'lover', 'pet', 'store', 'clerk', 'lott', 'schwartz', 'go', 'motion', 'marriag', 'despit', 'abl', 'earn', 'live', 'sole', 'puppet', 'craig', 'love', 'profess', 'allow', 'inhabit', 'skin', 'other', 'begin', 'take', 'abil', 'inhabit', 'skin', 'other', 'next', 'level', 'forc', 'take', 'job', 'file', 'clerk', 'kilter', 'lestercorp', 'locat', 'five', 'foot', 'tall', 'floor', 'manhattan', 'offic', 'build', 'behind', 'one', 'file', 'cabinet', 'work', 'area', 'craig', 'find', 'hidden', 'door', 'learn', 'portal', 'mind', 'john', 'malkovich', 'visit', 'portal', 'last', 'fifteen', 'minut', 'person', 'spit', 'ditch', 'next', 'new', 'jersey', 'turnpik', 'craig', 'fascin', 'mean', 'life', 'associ', 'find', 'lott', 'trip', 'portal', 'make', 'evalu', 'self', 'confid', 'maxin', 'lund', 'one', 'craig', 'worker', 'tell', 'portal', 'attract', 'think', 'money', 'make', 'opportun', 'sell', 'trip', 'malkovich', 'mind', 'offic', 'hour', '200', 'visit', 'craig', 'lott', 'maxin', 'begin', 'understand', 'anyon', 'enter', 'portal', 'abil', 'control', 'malkovich', 'mind', 'also', 'alter', 'entir', 'experi', 'make', 'maxin', 'fall', 'love', 'composit', 'abil', 'control', 'malkovich', 'mind', 'beg', 'question', 'ultim', 'psychedel', 'trip', 'malkovich', 'begin', 'feel', 'someth', 'right', 'world', 'know'], ['nameless', 'first', 'person', 'narrat', 'edward', 'norton', 'attend', 'support', 'group', 'attempt', 'subdu', 'emot', 'state', 'reliev', 'insomniac', 'state', 'meet', 'marla', 'helena', 'bonham', 'carter', 'anoth', 'fake', 'attende', 'support', 'group', 'life', 'seem', 'becom', 'littl', 'bearabl', 'howev', 'associ', 'tyler', 'brad', 'pitt', 'drag', 'underground', 'fight', 'club', 'soap', 'make', 'scheme', 'togeth', 'two', 'men', 'spiral', 'control', 'engag', 'competit', 'rivalri', 'love', 'power', 'narrat', 'expos', 'hidden', 'agenda', 'tyler', 'fight', 'club', 'must', 'accept', 'aw', 'truth', 'tyler', 'may', 'say'], ['eccentr', 'actor', 'daniel', 'hillard', 'amus', 'care', 'father', 'disastr', 'birthday', 'parti', 'son', 'daniel', 'wife', 'miranda', 'draw', 'line', 'file', 'divorc', 'see', 'three', 'children', 'week', 'sit', 'well', 'daniel', 'also', 'hold', 'job', 'studio', 'ship', 'clerk', 'recommend', 'liason', 'miranda', 'put', 'housekeep', 'daniel', 'take', 'upon', 'make', 'disguis', 'scottish', 'ladi', 'name', 'mrs', 'doubtfir', 'daniel', 'must', 'also', 'deal', 'miranda', 'new', 'boyfriend', 'stu', 'dunemy'], ['new', 'disc', 'jockey', 'ship', 'crete', 'vietnam', 'bring', 'humor', 'arm', 'forc', 'radio', 'turn', 'studio', 'ear', 'becom', 'wild', 'popular', 'troop', 'run', 'afoul', 'middl', 'manag', 'think', 'enough', 'air', 'tri', 'meet', 'vietnames', 'especi', 'girl', 'begin', 'brush', 'real', 'war', 'never', 'appear', 'radio'], ['death', 'row', 'guard', 'penitentiari', '1930', 'moral', 'dilemma', 'job', 'discov', 'one', 'prison', 'convict', 'murder', 'special', 'gift'], ['ultim', 'western', 'spoof', 'town', 'everyon', 'seem', 'name', 'johnson', 'way', 'railroad', 'order', 'grab', 'land', 'hedley', 'lemar', 'harvey', 'korman', 'polit', 'connect', 'nasti', 'person', 'send', 'henchmen', 'make', 'town', 'unliv', 'sheriff', 'kill', 'town', 'demand', 'new', 'sheriff', 'governor', 'mel', 'brook', 'hedley', 'convinc', 'send', 'town', 'first', 'black', 'sheriff', 'cleavon', 'littl', 'west', 'bart', 'sophist', 'urbanit', 'difficulti', 'win', 'townspeopl'], ['robbi', 'hart', 'sing', 'hit', '1980s', 'wed', 'celebr', 'also', 'keep', 'parti', 'go', 'good', 'spirit', 'know', 'say', 'say', 'julia', 'waitress', 'event', 'robbi', 'perform', 'find', 'someon', 'marri', 'prepar', 'wed', 'becom', 'clear', 'chosen', 'wrong', 'partner'], ['former', 'marri', 'petroleum', 'engin', 'still', 'issu', 'work', 'draft', 'assist', 'gung', 'navi', 'seal', 'top', 'secret', 'recoveri', 'oper', 'nuclear', 'sub', 'ambush', 'sunk', 'mysteri', 'circumst', 'deepest', 'water', 'earth'], ['success', 'clean', 'dodg', 'citi', 'wyatt', 'earp', 'move', 'tombston', 'arizona', 'wish', 'get', 'rich', 'obscur', 'meet', 'brother', 'well', 'old', 'friend', 'doc', 'holliday', 'band', 'outlaw', 'call', 'cowboy', 'caus', 'problem', 'region', 'various', 'act', 'random', 'violenc', 'inevit', 'come', 'confront', 'holliday', 'earp', 'lead', 'shoot', 'corral'], ['protagonist', 'alex', 'delarg', 'ultraviol', 'youth', 'futurist', 'britain', 'luck', 'eventu', 'run', 'arrest', 'convict', 'murder', 'rape', 'prison', 'alex', 'learn', 'experiment', 'program', 'convict', 'program', 'detest', 'violenc', 'goe', 'program', 'sentenc', 'reduc', 'back', 'street', 'sooner', 'expect', 'alex', 'ordeal', 'far', 'hit', 'mean', 'street', 'britain', 'hand', 'creat'], ['sibl', 'deal', 'target', 'question', 'tackl', 'movi', 'blue', 'collar', 'mickey', 'drive', 'new', 'york', 'taxicab', 'sinc', 'breakup', 'promiscu', 'fiance', 'heather', 'two', 'year', 'ago', 'younger', 'white', 'collar', 'brother', 'franci', 'cannot', 'let', 'mickey', 'forget', 'tragedi', 'hairi', 'ass', 'mickey', 'imag', 'apart', 'floor', 'guy', 'sex', 'heather', 'walk', 'find', 'relief', 'drive', 'cab', 'mickey', 'meet', 'art', 'student', 'name', 'hope', 'marri', 'know', 'hour', 'mickey', 'also', 'meet', 'old', 'lover', 'heather', 'learn', 'life', 'taxi', 'fare', 'cours', 'summer', 'franci', 'young', 'wall', 'street', 'corpor', 'raider', 'unhappi', 'marriag', 'rene', 'led', 'infidel', 'continu', 'shot', 'mickey', 'throughout', 'film', 'find', 'plot', 'devic', 'lend', 'humor', 'lesson', 'marriag', 'brotherhood', 'meet', 'start', 'danger', 'affair', 'heather', 'despit', 'mickey', 'warn', 'heather', 'gold', 'dig', 'nymphomaniac', 'goe', 'sexual', 'partner', 'often', 'person', 'chang', 'cloth', 'given', 'mickey', 'frame', 'refer', 'past', 'bride', 'hour', 'wonder', 'two', 'brother', 'along', 'father', 'ego', 'eccentr', 'emot', 'bereft', 'bigot', 'hard', 'core', 'chauvinist', 'allow', 'women', 'aboard', 'fish', 'boat', 'learn', 'strength', 'women', 'live'], ['dant', 'hick', 'good', 'day', 'work', 'clerk', 'small', 'conveni', 'store', 'told', 'come', 'work', 'day', 'dant', 'think', 'life', 'seri', 'end', 'day', 'prove', 'differ', 'read', 'newspap', 'girlfriend', 'caitlin', 'get', 'marri', 'present', 'girlfriend', 'reveal', 'somewhat', 'experi', 'sex', 'ever', 'imagin', 'princip', 'concern', 'hockey', 'game', 'afternoon', 'wake', 'friend', 'die', 'buddi', 'randal', 'grave', 'work', 'clerk', 'video', 'store', 'next', 'hate', 'job', 'much', 'dant', 'hate'], ['dorothi', 'gale', 'recent', 'come', 'home', 'kansa', 'land', 'almost', 'back', 'perfect', 'health', 'sinc', 'incid', 'tornado', 'cannot', 'get', 'wonder', 'place', 'head', 'frequent', 'talk', 'cannot', 'get', 'sleep', 'night', 'aunt', 'worri', 'health', 'well', 'think', 'suffer', 'delusion', 'depress', 'acut', 'insomnia', 'decid', 'take', 'see', 'special', 'doctor', 'anoth', 'town', 'tri', 'treat', 'electro', 'shock', 'treatment', 'take', 'nasti', 'dream', 'away', 'head', 'rescu', 'mysteri', 'girl', 'lead', 'back', 'new', 'adventur'], ['base', 'hit', 'seri', 'jim', 'phelp', 'sent', 'pragu', 'mission', 'prevent', 'theft', 'classifi', 'materi', 'wife', 'clair', 'trust', 'partner', 'ethan', 'hunt', 'member', 'phelp', 'team', 'unfortun', 'someth', 'went', 'horribl', 'wrong', 'mission', 'fail', 'leav', 'ethan', 'hunt', 'lone', 'survivor', 'report', 'fail', 'mission', 'kettridg', 'head', 'agenc', 'suspect', 'ethan', 'culprit', 'fail', 'mission', 'ethan', 'use', 'unorthodox', 'method', 'includ', 'aid', 'arm', 'dealer', 'go', 'name', 'max', 'tri', 'find', 'set', 'clear', 'name'], ['weather', 'man', 'reluct', 'sent', 'cover', 'stori', 'weather', 'forecast', 'rat', 'call', 'fourth', 'year', 'stori', 'make', 'effort', 'hide', 'frustrat', 'awak', 'follow', 'day', 'discov', 'groundhog', 'day', 'first', 'use', 'advantag', 'come', 'realis', 'doom', 'spend', 'rest', 'etern', 'place', 'see', 'peopl', 'thing', 'everi', 'day'], ['evil', 'trade', 'feder', 'led', 'nute', 'gunray', 'plan', 'take', 'peac', 'world', 'naboo', 'jedi', 'knight', 'qui', 'gon', 'jinn', 'obi', 'wan', 'kenobi', 'sent', 'confront', 'leader', 'everyth', 'goe', 'plan', 'two', 'jedi', 'escap', 'along', 'new', 'gungan', 'friend', 'jar', 'jar', 'bink', 'head', 'naboo', 'warn', 'queen', 'amidala', 'droid', 'alreadi', 'start', 'captur', 'naboo', 'queen', 'safe', 'eventu', 'land', 'tatooin', 'becom', 'friend', 'young', 'boy', 'known', 'anakin', 'skywalk', 'qui', 'gon', 'curious', 'boy', 'see', 'bright', 'futur', 'group', 'must', 'find', 'way', 'get', 'corusc', 'final', 'solv', 'trade', 'disput', 'someon', 'els', 'hide', 'shadow', 'sith', 'realli', 'extinct', 'queen', 'realli', 'say', 'special', 'young', 'boy'], ['michael', 'crocodil', 'dunde', 'australian', 'crocodil', 'hunter', 'live', 'australian', 'outback', 'run', 'safari', 'busi', 'trust', 'friend', 'mentor', 'walter', 'reilli', 'surviv', 'crocodil', 'attack', 'new', 'york', 'journalist', 'name', 'sue', 'arriv', 'interview', 'mick', 'surviv', 'learn', 'crocodil', 'hunter', 'save', 'sue', 'crocodil', 'sue', 'invit', 'mick', 'visit', 'new', 'york', 'citi', 'sinc', 'mick', 'never', 'citi', 'mick', 'find', 'cultur', 'life', 'new', 'york', 'citi', 'lot', 'differ', 'home', 'find', 'fall', 'love', 'sue'], ['taylor', 'two', 'astronaut', 'come', 'deep', 'hibern', 'find', 'ship', 'crash', 'escap', 'littl', 'cloth', 'find', 'land', 'planet', 'men', 'pre', 'lingual', 'uncivil', 'ape', 'learn', 'speech', 'technolog', 'taylor', 'captur', 'taken', 'citi', 'ape', 'damag', 'throat', 'silent', 'cannot', 'communic', 'ape'], ['jame', 'bond', '007', 'britain', 'top', 'agent', 'excit', 'mission', 'solv', 'mysteri', 'murder', 'fellow', 'agent', 'task', 'send', 'jamaica', 'join', 'forc', 'quarrel', 'loyal', 'cia', 'agent', 'felix', 'leiter', 'dodg', 'tarantula', 'fire', 'breath', 'dragon', 'trio', 'assassin', 'known', 'three', 'blind', 'mice', 'bond', 'meet', 'beauti', 'honey', 'ryder', 'goe', 'face', 'face', 'evil'], ['josh', 'tiffani', 'togeth', 'sinc', 'kid', 'enter', 'beth', 'sexi', 'blond', 'go', 'colleg', 'josh', 'one', 'night', 'josh', 'beth', 'hook', 'make', 'videotap', 'day', 'josh', 'plan', 'mail', 'video', 'tiffani', 'talk', 'colleg', 'life', 'wrong', 'tape', 'get', 'sent', 'forc', 'josh', 'along', 'pal', 'austin', 'tiffani', 'stop', 'see', 'tape', 'full', 'laugh', 'sure', 'enjoy', 'road', 'trip'], ['jame', 'bond', 'continu', 'fourth', 'mission', 'aim', 'recov', 'two', 'stolen', 'warhead', 'taken', 'evil', 'spectr', 'organ', 'world', 'held', 'hostag', 'bond', 'head', 'nassau', 'meet', 'beauti', 'domino', 'forc', 'thrill', 'confront', 'spectr', 'agent', 'emilio', 'largo', 'board', 'boat', 'disco', 'volant', '007', 'prevent', 'kill', 'million', 'innoc', 'victim'], ['new', 'film', 'current', 'product', 'killer', 'loos', 'murder', 'draw', 'report', 'cop', 'young', 'woman', 'set', 'movi', 'inspir', 'life', 'soon', 'find', 'deal', 'trilog', 'trilog', '...', 'anyth', 'happen'], ['alex', 'cross', 'washington', 'cop', 'forens', 'psychologist', 'learn', 'niec', 'go', 'colleg', 'north', 'carolina', 'miss', 'goe', 'learn', 'polic', 'think', 'among', 'victim', 'someon', 'kidnap', 'young', 'girl', 'hold', 'captiv', 'kill', 'dub', 'cassanova', 'great', 'lover', 'later', 'kate', 'one', 'victim', 'escap', 'tri', 'help', 'cross', 'find', 'niec'], ['dude', 'lebowski', 'mistaken', 'millionair', 'lebowski', 'two', 'thug', 'urin', 'rug', 'coerc', 'pay', 'debt', 'know', 'noth', 'attempt', 'gain', 'recompens', 'ruin', 'rug', 'wealthi', 'counterpart', 'accept', 'one', 'time', 'job', 'high', 'pay', 'enlist', 'help', 'bowl', 'buddi', 'walter', 'gun', 'tote', 'jewish', 'convert', 'anger', 'issu', 'decept', 'lead', 'troubl', 'soon', 'seem', 'everyon', 'porn', 'empir', 'tycoon', 'nihilist', 'want', 'someth', 'dude'], ['stori', 'mike', 'guy', 'left', 'girl', 'new', 'york', 'came', 'star', 'six', 'month', 'sinc', 'girlfriend', 'left', 'good', 'pal', 'friend', 'tri', 'get', 'back', 'social', 'scene', 'forget', 'year', 'relationship'], ['rocki', 'balboa', 'struggl', 'boxer', 'tri', 'make', 'big', 'time', 'work', 'debt', 'collector', 'pittanc', 'heavyweight', 'champion', 'apollo', 'creed', 'visit', 'philadelphia', 'manag', 'want', 'set', 'exhibit', 'match', 'creed', 'struggl', 'boxer', 'tout', 'fight', 'chanc', 'nobodi', 'becom', 'somebodi', 'match', 'suppos', 'easili', 'creed', 'someon', 'forgot', 'tell', 'rocki', 'see', 'shot', 'big', 'time'], ['luke', 'skywalk', 'han', 'solo', 'princess', 'leia', 'chewbacca', 'face', 'attack', 'imperi', 'forc', 'walker', 'ice', 'planet', 'hoth', 'han', 'leia', 'escap', 'millennium', 'falcon', 'luke', 'travel', 'dagobah', 'search', 'yoda', 'jedi', 'master', 'help', 'luke', 'surviv', 'dark', 'side', 'forc', 'beckon', 'ultim', 'duel', 'darth', 'vader'], ['open', 'alli', 'invas', 'normandi', 'june', '1944', 'member', '2nd', 'ranger', 'battalion', 'cpt', 'miller', 'fight', 'ashor', 'secur', 'beachhead', 'amidst', 'fight', 'two', 'brother', 'kill', 'action', 'earlier', 'new', 'guinea', 'third', 'brother', 'kia', 'mother', 'mrs', 'ryan', 'receiv', 'three', 'grave', 'telegram', 'day', 'unit', 'state', 'armi', 'chief', 'staff', 'georg', 'marshal', 'given', 'opportun', 'allevi', 'grief', 'learn', 'fourth', 'brother', 'privat', 'jame', 'ryan', 'decid', 'send', 'men', 'cpt', 'miller', 'select', 'member', '2nd', 'ranger', 'find', 'bring', 'back', 'home', 'mother', '...'], ['king', 'henri', 'england', 'insult', 'king', 'franc', 'result', 'lead', 'armi', 'battl', 'franc', 'along', 'way', 'young', 'king', 'must', 'struggl', 'sink', 'moral', 'troop', 'inner', 'doubt', 'war', 'culmin', 'bloodi', 'battl', 'agincourt'], ['tia', 'toni', 'two', 'orphan', 'youngster', 'extraordinari', 'power', 'luca', 'deranian', 'pose', 'uncl', 'order', 'get', 'kid', 'clutch', 'deranian', 'megaloman', 'boss', 'evil', 'millionair', 'aristotl', 'bolt', 'want', 'exploit', 'jason', 'cynic', 'widow', 'help', 'tia', 'toni', 'escap', 'witch', 'mountain', 'time', 'tia', 'toni', 'help', 'jason', 'escap', 'pain', 'loss', 'wife'], ['new', 'york', 'owner', 'sophist', 'antiqu', 'shop', 'russel', 'edwin', 'nash', 'challeng', 'sword', 'fight', 'park', 'lot', 'madison', 'squar', 'garden', 'man', 'call', 'iman', 'fasil', 'behead', 'russel', 'hide', 'sword', 'arrest', 'polic', 'leav', 'stadium', 'russel', 'recal', 'life', 'sixteenth', 'centuri', 'scotland', 'connor', 'macleod', 'dead', 'wound', 'battl', 'anoth', 'clan', 'howev', 'surpris', 'surviv', 'clan', 'believ', 'pact', 'devil', 'expel', 'land', 'meet', 'juan', 'sanchez', 'villa', 'lobo', 'ramirez', 'explain', 'immort', 'unless', 'behead', 'immort', 'disput', 'game', 'kill', 'end', 'one', 'surviv', 'receiv', 'price', 'power', 'immort', 'russel', 'releas', 'polic', 'snoopi', 'forens', 'agent', 'brenda', 'wyatt', 'attract', 'case', 'sinc', 'found', 'fragment', 'ancient', 'katana', 'follow', 'russel', 'also', 'immort', 'kurgan', 'hunt', 'macleod', 'brenda', 'middl', 'battl'], ['base', 'belov', 'children', 'novel', 'white', 'stori', 'littl', 'pig', 'name', 'wilbur', 'born', 'runt', 'arabl', 'decid', 'kill', 'dinner', 'food', 'daughter', 'fern', 'beg', 'let', 'live', 'greater', 'matur', 'wilbut', 'sold', 'fern', 'uncl', 'homer', 'zuckerman', 'whose', 'barnyard', 'left', 'yearn', 'friendship', 'snub', 'barn', 'anim', 'wilbur', 'scare', 'end', 'season', 'know', 'come', 'time', 'end', 'dinner', 'tabl', 'hatch', 'plan', 'charlott', 'gentl', 'wise', 'grey', 'spider', 'live', 'pen', 'ensur', 'never', 'happen'], ['young', 'neurosurgeon', 'gene', 'wilder', 'inherit', 'castl', 'grandfath', 'famous', 'victor', 'von', 'frankenstein', 'castl', 'find', 'funni', 'hunchback', 'call', 'igor', 'pretti', 'lab', 'assist', 'name', 'inga', 'old', 'housekeep', 'frau', 'blucher', 'iiiiihhh', '!-.', 'young', 'frankenstein', 'believ', 'work', 'grandfath', 'crap', 'discov', 'book', 'mad', 'doctor', 'describ', 'reanim', 'experi', 'sudden', 'chang', 'mind', '...'], ['thoma', 'anderson', 'man', 'live', 'two', 'live', 'day', 'averag', 'comput', 'programm', 'night', 'hacker', 'known', 'neo', 'neo', 'alway', 'question', 'realiti', 'truth', 'far', 'beyond', 'imagin', 'neo', 'find', 'target', 'polic', 'contact', 'morpheus', 'legendari', 'comput', 'hacker', 'brand', 'terrorist', 'govern', 'morpheus', 'awaken', 'neo', 'real', 'world', 'ravag', 'wasteland', 'human', 'captur', 'race', 'machin', 'live', 'human', 'bodi', 'heat', 'electrochem', 'energi', 'imprison', 'mind', 'within', 'artifici', 'realiti', 'known', 'matrix', 'rebel', 'machin', 'neo', 'must', 'return', 'matrix', 'confront', 'agent', 'super', 'power', 'comput', 'program', 'devot', 'snuf', 'neo', 'entir', 'human', 'rebellion'], ['stanley', 'goodspe', 'live', 'washington', 'biochemist', 'work', 'fbi', 'soon', 'fiance', 'carla', 'pestalozzi', 'announc', 'pregnant', 'stanley', 'get', 'call', 'fbi', 'director', 'jame', 'womack', 'womack', 'tell', 'stanley', 'san', 'francisco', 'alcatraz', 'island', 'taken', 'hostag', 'along', 'tourist', 'marin', 'general', 'franci', 'xavier', 'hummel', 'year', 'protest', 'govern', 'refus', 'pay', 'benefit', 'famili', 'war', 'veteran', 'die', 'covert', 'militari', 'oper', 'death', 'wife', 'barbara', 'hummel', 'march', '1995', 'drove', 'general', 'hummel', 'edg', 'hold', 'hostag', 'order', 'get', 'point', 'across', 'stanley', 'need', 'general', 'hummel', 'stolen', 'gas', 'warhead', 'announc', 'launch', 'onto', 'san', 'francisco', 'unless', 'demand', 'met', 'stanley', 'know', 'disarm', 'bomb', 'stanley', 'need', 'someon', 'know', 'alcatraz', 'well', 'enough', 'get', 'insid', 'man', 'former', 'british', 'intellig', 'agent', 'john', 'patrick', 'mason', 'prison', 'past', 'year', 'without', 'trial', 'accus', 'steal', 'privat', 'file', 'edgar', 'hoover', '1962', 'john', 'becam', 'inmat', 'ever', 'escap', 'alcatraz', 'stay', 'alcatraz', 'long', 'enough', 'father', 'daughter', 'name', 'jade', 'angelou', 'despit', 'extrem', 'hatr', 'fbi', 'john', 'agre', 'help', 'stanley', 'john', 'stanley', 'sent', 'alcatraz', 'navi', 'seal', 'team', 'general', 'hummel', 'men', 'kill', 'entir', 'seal', 'team', 'leav', 'stanley', 'john', 'work', 'rescu', 'hostag'], ['harrow', 'ride', 'carpathian', 'mountain', 'eastern', 'europ', 'renfield', 'enter', 'castl', 'dracula', 'final', 'transferr', 'carfax', 'abbey', 'london', 'count', 'dracula', 'actual', 'vampir', 'renfield', 'drug', 'eerili', 'hypnot', 'count', 'turn', 'one', 'thrall', 'protect', 'sea', 'voyag', 'london', 'suck', 'blood', 'turn', 'young', 'luci', 'weston', 'vampir', 'dracula', 'turn', 'attent', 'friend', 'mina', 'seward', 'daughter', 'seward', 'call', 'specialist', 'van', 'hels', 'diagnos', 'sudden', 'deterior', 'mina', 'health', 'van', 'hels', 'realiz', 'dracula', 'inde', 'vampir', 'tri', 'prepar', 'mina', 'fianc', 'john', 'harker', 'seward', 'come', 'measur', 'taken', 'prevent', 'mina', 'becom', 'one', 'undead'], ['stork', 'deliv', 'babi', 'eleph', 'mrs', 'jumbo', 'veteran', 'circus', 'newborn', 'ridicul', 'truli', 'enorm', 'ear', 'dub', 'dumbo', 'separ', 'mother', 'dumbo', 'releg', 'circus', 'clown', 'act', 'friend', 'mous', 'assist', 'dumbo', 'achiev', 'full', 'potenti'], ['death', 'sometim', 'forti', 'third', 'year', 'suburbanit', 'lester', 'burnham', 'tell', 'last', 'week', 'life', 'idea', 'immin', 'pass', 'husband', 'real', 'estat', 'agent', 'carolyn', 'burnham', 'father', 'high', 'school', 'student', 'jani', 'burnham', 'although', 'lester', 'carolyn', 'love', 'mere', 'toler', 'typic', 'wallflow', 'jani', 'hate', 'parent', 'three', 'suffer', 'individu', 'silenc', 'home', 'life', 'jani', 'tri', 'steer', 'clear', 'parent', 'carolyn', 'relat', 'new', 'real', 'estat', 'busi', 'want', 'creat', 'persona', 'success', 'career', 'aspir', 'profession', 'life', 'buddi', 'kane', 'king', 'real', 'estat', 'busi', 'neighborhood', 'lester', 'mere', 'walk', 'mindless', 'life', 'includ', 'job', 'advertis', 'compani', 'downsiz', 'like', 'employe', 'justifi', 'posit', 'newli', 'hire', 'effici', 'expert', 'keep', 'job', 'thing', 'chang', 'lester', 'fall', 'love', 'first', 'sight', 'jani', 'experienc', 'classmat', 'angela', 'hay', 'jani', 'angela', 'see', 'lester', 'sexual', 'infatu', 'angela', 'court', 'attent', 'man', 'sign', 'model', 'materi', 'appear', 'seventeen', 'career', 'aspir', 'lester', 'infatu', 'angela', 'give', 'reenerg', 'view', 'life', 'open', 'care', 'anymor', 'anyon', 'think', 'anyon', 'except', 'angela', 'infatu', 'coincid', 'fitts', 'move', 'next', 'door', 'homophob', 'disciplinarian', 'marin', 'colonel', 'frank', 'fitt', 'rule', 'hous', 'militari', 'fist', 'fist', 'figur', 'liter', 'semi', 'comatos', 'wife', 'barbara', 'fitt', 'bright', 'quiet', 'subvers', 'eighteen', 'year', 'old', 'son', 'ricki', 'fitt', 'open', 'abid', 'father', 'rule', 'behind', 'scene', 'live', 'quit', 'differ', 'perspect', 'much', 'like', 'lester', 'infatu', 'ricki', 'immedi', 'becom', 'infatu', 'jani', 'consid', 'girl', 'like', 'angela', 'ordinari', 'entri', 'angela', 'fitts', 'burnham', 'live', 'ultim', 'lead', 'player', 'confront', 'truli', 'heart'], ['jame', 'bond', '007', 'search', 'russian', 'decod', 'machin', 'known', 'lektor', 'bond', 'need', 'find', 'machin', 'evil', 'spectr', 'organ', 'discov', 'first', 'whilst', 'romant', 'link', 'russian', 'girl', 'tatiana', 'romanova', 'bond', 'sneak', 'way', 'around', 'istanbul', 'whilst', 'spectr', 'agent', 'tri', 'pick', 'includ', 'power', 'donald', 'red', 'grant', 'kgb', 'agent', 'rosa', 'klebb', 'know', 'trick', 'book', 'even', 'possess', 'incred', 'poison', 'tip', 'shoe'], ['imperi', 'forc', 'order', 'cruel', 'darth', 'vader', 'hold', 'princess', 'leia', 'hostag', 'effort', 'quell', 'rebellion', 'galact', 'empir', 'luke', 'skywalk', 'han', 'solo', 'captain', 'millennium', 'falcon', 'work', 'togeth', 'companion', 'droid', 'duo', '3po', 'rescu', 'beauti', 'princess', 'help', 'rebel', 'allianc', 'restor', 'freedom', 'justic', 'galaxi'], ['team', 'special', 'forc', 'op', 'led', 'tough', 'fair', 'soldier', 'major', 'dutch', 'schaefer', 'order', 'assist', 'cia', 'man', 'colonel', 'dillon', 'rescu', 'mission', 'potenti', 'survivor', 'helicopt', 'down', 'remot', 'south', 'american', 'jungl', 'long', 'land', 'dutch', 'team', 'discov', 'sent', 'fals', 'pretens', 'decept', 'turn', 'least', 'worri', 'though', 'find', 'method', 'hunt', 'someth', 'world'], ['1950', 'los', 'angel', 'seedi', 'backdrop', 'intric', 'noir', 'ish', 'tale', 'polic', 'corrupt', 'hollywood', 'sleaz', 'three', 'differ', 'cop', 'truth', 'style', 'exley', 'golden', 'boy', 'polic', 'forc', 'will', 'almost', 'anyth', 'get', 'ahead', 'except', 'sell', 'bud', 'white', 'readi', 'break', 'rule', 'seek', 'justic', 'bare', 'abl', 'keep', 'rage', 'violenc', 'control', 'jack', 'vincenn', 'alway', 'look', 'celebr', 'quick', 'buck', 'conscienc', 'drive', 'join', 'exley', 'white', 'one', 'way', 'path', 'find', 'truth', 'behind', 'dark', 'world', 'crime'], ['novemb', '1963', 'presid', 'john', 'kennedi', 'assassin', 'dalla', 'lee', 'harvey', 'oswald', 'arrest', 'crime', 'subsequ', 'shot', 'jack', 'rubi', 'suppos', 'aveng', 'presid', 'death', 'investig', 'conclud', 'lee', 'harvey', 'oswald', 'jack', 'rubi', 'act', 'alon', 'respect', 'crime', 'louisiana', 'district', 'attorney', 'jim', 'garrison', 'skeptic', 'assembl', 'trust', 'group', 'peopl', 'garrison', 'conduct', 'investig', 'bring', 'backlash', 'power', 'govern', 'polit', 'figur'], ['william', 'wallac', 'scottish', 'rebel', 'lead', 'upris', 'cruel', 'english', 'ruler', 'edward', 'longshank', 'wish', 'inherit', 'crown', 'scotland', 'young', 'boy', 'william', 'wallac', 'father', 'brother', 'along', 'mani', 'other', 'lost', 'live', 'tri', 'free', 'scotland', 'lose', 'anoth', 'love', 'one', 'william', 'wallac', 'begin', 'long', 'quest', 'make', 'scotland', 'free', 'along', 'assist', 'robert', 'bruce'], ['sir', 'robert', 'beaumont', 'behind', 'schedul', 'railroad', 'africa', 'enlist', 'note', 'engin', 'john', 'henri', 'patterson', 'right', 'ship', 'beaumont', 'expect', 'result', 'everyth', 'seem', 'great', 'crew', 'discov', 'mutil', 'corps', 'project', 'foreman', 'seem', 'kill', 'lion', 'sever', 'attack', 'patterson', 'call', 'fame', 'hunter', 'charl', 'remington', 'final', 'met', 'match', 'bloodthirsti', 'lion'], ['anim', 'stori', 'bambi', 'young', 'deer', 'hail', 'princ', 'forest', 'birth', 'bambi', 'grow', 'make', 'friend', 'anim', 'forest', 'learn', 'skill', 'need', 'surviv', 'even', 'find', 'love', 'one', 'day', 'howev', 'hunter', 'come', 'bambi', 'must', 'learn', 'brave', 'father', 'lead', 'deer', 'safeti'], ['cyborg', 'sent', 'futur', 'dead', 'mission', 'kill', 'sarah', 'connor', 'young', 'woman', 'whose', 'life', 'great', 'signific', 'year', 'come', 'sarah', 'one', 'protector', 'kyle', 'rees', 'also', 'sent', 'futur', 'termin', 'use', 'except', 'intellig', 'strength', 'find', 'sarah', 'way', 'stop', 'seem', 'indestruct', 'cyborg'], ['two', 'segment', 'look', 'effect', 'militari', 'mindset', 'war', 'vietnam', 'era', 'marin', 'first', 'half', 'follow', 'group', 'recruit', 'boot', 'camp', 'command', 'punish', 'gunneri', 'sergeant', 'hartman', 'second', 'half', 'show', 'one', 'recruit', 'joker', 'cover', 'war', 'correspond', 'star', 'stripe', 'focus', 'tet', 'offens'], ['armi', 'privat', 'witt', 'awol', 'found', 'imprison', 'troop', 'carrier', 'compani', 'first', 'sergeant', 'welsh', 'men', 'compani', '1st', 'battalion', '27th', 'infantri', 'regiment', '25th', 'infantri', 'divis', 'brought', 'guadalcan', 'reinforc', 'campaign', 'secur', 'henderson', 'field', 'seiz', 'island', 'japanes', 'arriv', 'near', 'hill', '210', 'key', 'japanes', 'posit', 'task', 'captur', 'hill', 'cost', 'happen', 'next', 'stori', 'develop', 'redempt', 'meaningless', 'war', 'regardless', 'outcom'], ['king', 'roland', 'planet', 'druidia', 'tri', 'marri', 'daughter', 'princess', 'vespa', 'princ', 'valium', 'vespa', 'kidnap', 'evil', 'race', 'spacebal', 'spacebal', 'ask', 'roland', 'tremend', 'ransom', 'air', 'druidia', 'see', 'air', 'spacebal', 'serious', 'pollut', 'problem', '...).', 'king', 'decid', 'offer', 'generous', 'amount', 'money', 'space', 'rogu', 'lone', 'starr', 'persuad', 'save', 'vespa', 'follow', 'parodi', '_lot_', 'famous', 'movi'], ['new', 'england', 'earli', '20th', 'centuri', 'pete', 'nine', 'year', 'old', 'orphan', 'escap', 'brutal', 'adopt', 'parent', 'gogan', 'friend', 'cartoon', 'dragon', 'name', 'elliott', 'success', 'escap', 'passamaquoddi', 'main', 'live', 'nora', 'lighthous', 'keeper', 'father', 'lampi', 'elliott', 'sought', 'medicin', 'purpos', 'corrupt', 'terminus'], ['tess', 'trueheart', 'want', 'settl', 'quiet', 'life', 'boyfriend', 'detect', 'dick', 'traci', 'someth', 'pretti', 'rotten', 'go', 'town', 'someon', 'pretti', 'rotten', 'behind', 'traci', 'hand', 'full', 'like', 'villain', 'big', 'boy', 'capric', 'almost', 'irresist', 'breathless', 'mahoney'], ['marti', 'mcfli', 'typic', 'american', 'teenag', 'eighti', 'accident', 'sent', 'back', '1955', 'plutonium', 'power', 'delorean', 'time', 'machin', 'invent', 'slight', 'mad', 'scientist', 'often', 'hyster', 'alway', 'amaz', 'trip', 'back', 'time', 'marti', 'must', 'make', 'certain', 'teenag', 'parent', 'meet', 'fall', 'love', 'get', 'back', 'futur'], ['upon', 'death', 'brother', 'larri', 'talbot', 'return', 'america', 'ancestr', 'home', 'wale', 'visit', 'gypsi', 'camp', 'villag', 'girl', 'jenni', 'william', 'attack', 'bela', 'gypsi', 'turn', 'werewolf', 'larri', 'kill', 'werewolf', 'bitten', 'fight', 'bela', 'mother', 'tell', 'caus', 'becom', 'werewolf', 'full', 'moon', 'larri', 'confess', 'plight', 'unbeliev', 'father', 'sir', 'john', 'join', 'villag', 'hunt', 'wolf', 'transform', 'full', 'moon', 'larri', 'head', 'forest', 'fate', 'meet', 'sir', 'john', 'gwen', 'conliff'], ['base', 'comic', 'book', 'unbeknownst', 'peopl', 'privat', 'agenc', 'code', 'name', 'mib', 'agenc', 'kind', 'extra', 'terrestri', 'surveil', 'corpor', 'one', 'agenc', 'finest', 'men', 'go', 'name', 'tommi', 'lee', 'jone', 'recruit', 'new', 'addit', 'agenc', 'chosen', 'jame', 'edward', 'smith', 'one', 'day', 'fli', 'saucer', 'crash', 'earth', 'alien', 'part', 'bug', 'race', 'take', 'bodi', 'farmer', 'vincent', 'onofrio', 'head', 'new', 'york', 'search', 'super', 'energi', 'sourc', 'call', 'galaxi', 'agent', 'must', 'stop', 'bug', 'escap', 'galaxi'], ['1950s', 'manhattan', 'lavatori', 'attend', 'tom', 'ripley', 'borrow', 'princeton', 'jacket', 'play', 'piano', 'garden', 'parti', 'wealthi', 'father', 'recent', 'princeton', 'grad', 'chat', 'tom', 'tom', 'pretend', 'know', 'son', 'soon', 'offer', '000', 'itali', 'convinc', 'dicki', 'greenleaf', 'return', 'home', 'itali', 'tom', 'attach', 'dicki', 'marg', 'dicki', 'cultur', 'fiancé', 'pretend', 'love', 'jazz', 'harbor', 'homoerot', 'hope', 'soak', 'luxuri', 'besid', 'lie', 'tom', 'talent', 'includ', 'impress', 'forgeri', 'handsom', 'confid', 'dicki', 'tire', 'tom', 'dismiss', 'bore', 'tom', 'goe', 'extrem', 'length', 'make', 'greenleaf', 'privileg'], ['harri', 'lloyd', 'two', 'good', 'friend', 'happen', 'realli', 'stupid', 'duo', 'set', 'cross', 'countri', 'trip', 'provid', 'aspen', 'colorado', 'return', 'briefcas', 'full', 'money', 'right', 'owner', 'beauti', 'woman', 'name', 'mari', 'swanson', 'trip', 'one', 'mishap', 'anoth', 'duo', 'eventu', 'make', 'aspen', 'two', 'soon', 'realiz', 'mari', 'briefcas', 'least', 'problem'], ['doctor', 'leo', 'marvin', 'egotist', 'psychotherapist', 'new', 'york', 'citi', 'look', 'forward', 'forthcom', 'appear', 'good', 'morn', 'america', 'telecast', 'plan', 'brag', 'babi', 'step', 'new', 'book', 'emot', 'disord', 'theori', 'detail', 'philosophi', 'treat', 'patient', 'phobia', 'meanwhil', 'bob', 'wiley', 'reclus', 'afraid', 'leav', 'apart', 'talk', 'door', 'bob', 'pawn', 'leo', 'psychotherapist', 'colleagu', 'bob', 'becom', 'attach', 'leo', 'leo', 'find', 'bob', 'extrem', 'annoy', 'leo', 'accompani', 'wife', 'fay', 'daughter', 'anna', 'son', 'siggi', 'peac', 'new', 'hampshir', 'lakesid', 'cottag', 'month', 'long', 'vacat', 'leo', 'think', 'freed', 'bob', 'leo', 'expect', 'mesmer', 'famili', 'prowess', 'brilliant', 'husband', 'remark', 'father', 'know', 'know', 'instruct', 'wife', 'rais', 'kid', 'bob', 'go', 'let', 'leo', 'enjoy', 'quiet', 'summer', 'lake', 'clever', 'trick', 'telephon', 'oper', 'doctor', 'exchang', 'bob', 'discov', 'whereabout', 'leo', 'famili', 'despit', 'phobia', 'travel', 'alon', 'bob', 'somehow', 'manag', 'talk', 'onto', 'bus', 'arriv', 'new', 'hampshir', 'leo', 'vacat', 'come', 'screech', 'halt', 'moment', 'see', 'bob', 'witti', 'person', 'abil', 'manipul', 'peopl', 'good', 'sens', 'humor', 'bob', 'quick', 'becom', 'annoy', 'leo', 'fay', 'anna', 'siggi', 'think', 'bob', 'fun', 'leo', 'dull', 'fear', 'lose', 'famili', 'bob', 'leo', 'frantic', 'tri', 'find', 'way', 'make', 'bob', 'back', 'new', 'york', 'easi', 'leo', 'hope', 'leo', 'find', 'step', 'outsid', 'law', 'tri', 'get', 'bob', 'stay', 'away', 'fay', 'anna', 'siggi', 'leo', 'slowli', 'goe', 'berserk', 'make', 'plan', 'kill', 'bob'], ['jack', 'ryan', 'back', 'time', 'bad', 'guy', 'govern', 'admir', 'jame', 'greer', 'becom', 'sick', 'cancer', 'ryan', 'appoint', 'act', 'cia', 'deputi', 'director', 'intellig', 'almost', 'draw', 'breath', 'new', 'posit', 'one', 'presid', 'closest', 'friend', 'famili', 'murder', 'sleep', 'appear', 'drug', 'cartel', 'ryan', 'call', 'investig', 'unknown', 'cia', 'alreadi', 'sent', 'secret', 'field', 'oper', 'lead', 'illeg', 'paramilitari', 'forc', 'colombia', 'cartel', 'thing', 'get', 'even', 'complic', 'team', 'set', 'lose', 'agent', 'field', 'friend', 'wife', 'murder', 'agent', 'secretari', 'murder', 'day', 'ryan', 'must', 'risk', 'career', 'life', 'expos', 'truth', 'behind', 'mysteri'], ['gentl', 'alien', 'becom', 'strand', 'earth', 'discov', 'befriend', 'young', 'boy', 'name', 'elliott', 'bring', 'extraterrestri', 'suburban', 'california', 'hous', 'elliott', 'introduc', 'alien', 'dub', 'brother', 'littl', 'sister', 'gerti', 'children', 'decid', 'keep', 'exist', 'secret', 'soon', 'howev', 'fall', 'ill', 'result', 'govern', 'intervent', 'dire', 'situat', 'elliott', 'alien'], ['victorian', 'london', 'england', 'littl', 'mous', 'girl', 'toymak', 'father', 'abduct', 'pegleg', 'bat', 'enlist', 'aid', 'basil', 'baker', 'street', 'rodent', 'world', 'answer', 'sherlock', 'holm', 'case', 'expand', 'basil', 'uncov', 'crime', 'link', 'plot', 'crown'], ['straight', 'page', 'pulp', 'comic', 'past', 'era', 'rocket', 'recreat', '1930', 'hollywood', 'complet', 'gangster', 'nazi', 'spi', 'growth', 'age', 'aviat', 'young', 'pilot', 'cliff', 'secord', 'stumbl', 'top', 'secret', 'rocket', 'pack', 'help', 'mechan', 'mentor', 'peevi', 'attempt', 'save', 'girl', 'stop', 'nazi', 'rocket'], ['set', '1935', 'professor', 'archaeologist', 'legendari', 'hero', 'name', 'indiana', 'jone', 'back', 'action', 'newest', 'adventur', 'time', 'team', 'night', 'club', 'singer', 'name', 'wilhelmina', 'willi', 'scott', 'twelv', 'year', 'old', 'boy', 'name', 'short', 'round', 'end', 'indian', 'small', 'distress', 'villag', 'peopl', 'believ', 'evil', 'spirit', 'taken', 'children', 'away', 'sacr', 'precious', 'stone', 'stolen', 'also', 'discov', 'great', 'mysteri', 'terror', 'surround', 'boobi', 'trap', 'templ', 'known', 'templ', 'doom', 'thugge', 'begin', 'attempt', 'rise', 'believ', 'power', 'five', 'sankara', 'stone', 'rule', 'world', 'indiana', 'put', 'end', 'thugge', 'campaign', 'rescu', 'lost', 'children', 'win', 'girl', 'conquer', 'templ', 'doom'], ['dramat', 'courtroom', 'thriller', 'daniel', 'kaffe', 'navi', 'lawyer', 'never', 'seen', 'insid', 'courtroom', 'defend', 'two', 'stubborn', 'marin', 'accus', 'murder', 'colleagu', 'kaffe', 'known', 'lazi', 'arrang', 'plea', 'bargain', 'downey', 'aunt', 'ginni', 'appoint', 'cmdr', 'galloway', 'repres', 'also', 'legal', 'staff', 'ltjg', 'sam', 'weinberg', 'team', 'round', 'mani', 'fact', 'kaffe', 'discov', 'realli', 'cut', 'trial', 'work', 'defens', 'origin', 'base', 'upon', 'fact', 'pfc', 'santiago', 'victim', 'given', 'code', 'red', 'santiago', 'basic', 'screw', 'gitmo', 'screw', 'up', 'toler', 'especi', 'col', 'nathan', 'jessup', 'cuba', 'jessup', 'two', 'senior', 'offic', 'tri', 'give', 'help', 'kaffe', 'know', 'someth', 'fishi', 'conclus', 'film', 'firework', 'set', 'confront', 'jessup', 'kaffe'], ['stori', 'nine', 'year', 'old', 'boy', 'name', 'hogarth', 'hugh', 'make', 'friend', 'innoc', 'alien', 'giant', 'robot', 'came', 'outer', 'space', 'meanwhil', 'paranoid', 'govern', 'agent', 'name', 'kent', 'mansley', 'arriv', 'town', 'determin', 'destroy', 'giant', 'cost', 'hogarth', 'protect', 'keep', 'dean', 'mccoppin', 'place', 'junkyard'], ['three', 'work', 'silent', 'movi', 'actor', 'accident', 'drawn', 'mexican', 'villag', 'harass', 'gang', 'outlaw', 'three', 'ned', 'lucki', 'day', 'dusti', 'bottom', 'play', 'lone', 'ranger', 'type', 'movi', 'must', 'play', 'part', 'real'], ['english', 'librarian', 'call', 'evelyn', 'carnahan', 'becom', 'interest', 'start', 'archaeolog', 'dig', 'ancient', 'citi', 'hamunaptra', 'gain', 'help', 'rick', 'connel', 'save', 'death', 'evelyn', 'brother', 'jonathan', 'rick', 'unawar', 'anoth', 'group', 'explor', 'interest', 'dig', 'unfortun', 'everyon', 'group', 'end', 'unleash', 'curs', 'laid', 'dead', 'high', 'priest', 'imhotep', 'mummi', 'awak', 'go', 'take', 'lot', 'gun', 'send', 'back', 'came'], ['four', 'boy', 'south', 'park', 'stan', 'marsh', 'kyle', 'stepbroth', 'ike', 'broflovski', 'eric', 'cartman', 'kenni', 'mccormick', 'see', 'rate', 'movi', 'featur', 'canadian', 'terranc', 'phillip', 'ass', 'fire', 'pronounc', 'corrupt', 'kyle', 'mom', 'sheila', 'rest', 'parent', 'pressur', 'unit', 'state', 'wage', 'war', 'canada', 'world', 'war', 'stan', 'kyle', 'cartman', 'save', 'terrenc', 'phillip', 'satan', 'lover', 'saddam', 'hussein', 'hell', 'rule', 'world', 'end', 'whole', 'world'], ['retir', 'lieuten', 'command', 'quinton', 'mchale', 'spend', 'day', 'putter', 'around', 'caribbean', 'old', 'sell', 'homebrew', 'ice', 'cream', 'swimsuit', 'calendar', 'brought', 'retir', 'old', 'nemesi', 'turn', 'second', 'best', 'terrorist', 'world', 'major', 'vladikov', 'take', 'island', 'san', 'moreno', 'start', 'build', 'nuclear', 'launch', 'silo', 'help', 'old', 'crew', 'hindranc', 'captain', 'wallac', 'binghampton', 'sank', 'cruis', 'liner', 'back', 'mchale', 'tri', 'put', 'vladikov', 'busi'], ['john', 'max', 'elder', 'men', 'live', 'next', 'door', 'continu', 'argu', 'insult', 'way', 'year', 'one', 'day', 'ariel', 'move', 'street', 'men', 'attract', 'rivalri', 'step', 'gear'], ['nichola', 'van', 'orton', 'wealthi', 'san', 'francisco', 'banker', 'absolut', 'loner', 'even', 'spend', 'birthday', 'alon', 'year', '48th', 'birthday', 'age', 'father', 'commit', 'suicid', 'brother', 'conrad', 'gone', 'long', 'ago', 'surrend', 'addict', 'kind', 'sudden', 'return', 'give', 'nichola', 'card', 'give', 'entri', 'unusu', 'entertain', 'provid', 'someth', 'call', 'consum', 'recreat', 'servic', 'crs', 'give', 'curios', 'nichola', 'visit', 'crs', 'kind', 'weird', 'bad', 'thing', 'start', 'happen'], ['personnel', '4077', 'mash', 'unit', 'deal', 'horror', 'korean', 'war', 'stress', 'face', 'surgeri', 'whatev', 'mean', 'tone', 'mash', 'establish', 'recent', 'arriv', 'surgeon', 'captain', 'hawkey', 'pierc', 'duke', 'forrest', 'trapper', 'john', 'mcintyr', 'latter', 'hawkey', 'know', 'met', 'somewher', 'trapper', 'divulg', 'whose', 'antic', 'best', 'describ', 'non', 'regul', 'negat', 'word', 'one', 'fellow', 'mash', 'er', 'unmilitari', 'unit', 'command', 'offic', 'colonel', 'henri', 'blake', 'care', 'behavior', 'long', 'affect', 'long', 'job', 'well', 'behavior', 'extrem', 'bother', 'fellow', 'surgeon', 'major', 'frank', 'burn', 'recent', 'arriv', 'head', 'nurs', 'major', 'margaret', 'houlihan', 'obtain', 'nicknam', 'hot', 'lip', 'base', 'inform', 'glean', 'underhand', 'mean', 'beyond', 'battl', 'frank', 'hot', 'lip', 'hawkey', 'duke', 'trapper', 'help', 'unit', 'dentist', 'painless', 'person', 'crisi', 'tri', 'figur', 'hot', 'lip', 'true', 'blond', 'travel', 'japan', 'work', 'hope', 'recreat', 'trip', 'hope', 'win', 'big', 'footbal', 'bet', 'anoth', 'unit', 'surfac', 'underdog'], ['south', 'florida', 'high', 'school', 'counselor', 'accus', 'rape', 'manipul', 'rich', 'girl', 'trailer', 'trash', 'classmat', 'cop', 'case', 'begin', 'suspect', 'conspiraci', 'dive', 'elabor', 'devious', 'web', 'greed', 'betray', 'find', 'truth'], ['teenag', 'sarah', 'forc', 'father', 'stepmoth', 'babysit', 'babi', 'brother', 'tobi', 'outsid', 'home', 'tobi', 'stop', 'cri', 'sarah', 'wish', 'brother', 'taken', 'goblin', 'king', 'blue', 'tobi', 'stop', 'cri', 'sarah', 'look', 'cradl', 'learn', 'wish', 'grant', 'goblin', 'king', 'jarethha', 'taken', 'castl', 'goblin', 'citi', 'middl', 'labyrinth', 'sarah', 'repent', 'ask', 'jareth', 'give', 'tobi', 'back', 'goblin', 'king', 'tell', 'rescu', 'brother', 'midnight', 'otherwis', 'tobi', 'turn', 'goblin', 'soon', 'sarah', 'team', 'coward', 'goblin', 'hoggl', 'beast', 'ludo', 'knight', 'didymus', 'dog', 'ambrosius', 'journey', 'rescu', 'tobi', 'time'], ['follow', 'truck', 'hijack', 'new', 'york', 'five', 'conmen', 'arrest', 'brought', 'togeth', 'question', 'none', 'guilti', 'plan', 'reveng', 'oper', 'polic', 'oper', 'goe', 'well', 'influenc', 'legendari', 'mastermind', 'crimin', 'call', 'keyser', 'soze', 'felt', 'becom', 'clear', 'one', 'wrong', 'soze', 'point', 'must', 'pay', 'back', 'payback', 'job', 'leav', 'men', 'dead', 'boat', 'explos', 'real', 'question', 'aris', 'actual', 'keyser', 'soze'], ['year', 'old', 'boy', 'goe', 'miss', '1978', 'reappear', '1986', 'eight', 'year', 'pass', 'age', 'coincid', 'time', 'come', 'back', 'fli', 'saucer', 'found', 'entangl', 'power', 'line'], ['phoenix', 'officework', 'marion', 'crane', 'fed', 'way', 'life', 'treat', 'meet', 'lover', 'sam', 'lunch', 'break', 'cannot', 'get', 'marri', 'sam', 'give', 'money', 'away', 'alimoni', 'one', 'friday', 'marion', 'trust', 'bank', '000', 'employ', 'see', 'opportun', 'take', 'money', 'start', 'new', 'life', 'marion', 'leav', 'town', 'head', 'toward', 'sam', 'california', 'store', 'tire', 'long', 'drive', 'caught', 'storm', 'get', 'main', 'highway', 'pull', 'bate', 'motel', 'motel', 'manag', 'quiet', 'young', 'man', 'call', 'norman', 'seem', 'domin', 'mother'], ['countri', 'freedonia', 'middl', 'financi', 'crisi', 'brink', 'revolut', 'order', 'gain', 'bail', 'wealthi', 'mrs', 'teasdal', 'govern', 'appoint', 'rufus', 'firefli', 'presid', 'howev', 'firefli', 'shun', 'pomp', 'pretenti', 'govern', 'along', 'prudenc', 'ration', 'meanwhil', 'neighbour', 'countri', 'sylvania', 'plot', 'overthrow', 'freedonia', 'send', 'pinki', 'chicolini', 'spi', 'firefli', 'war', 'seem', 'inevit'], ['art', 'collector', 'appeal', 'jone', 'embark', 'search', 'holi', 'grail', 'learn', 'anoth', 'archaeologist', 'disappear', 'search', 'precious', 'goblet', 'miss', 'man', 'father', 'henri', 'jone', 'artifact', 'much', 'harder', 'find', 'expect', 'power', 'much', 'impur', 'heart'], ['movi', 'tell', 'stori', 'rock', 'singer', 'pink', 'sit', 'hotel', 'room', 'los', 'angel', 'burnt', 'music', 'busi', 'abl', 'perform', 'stage', 'help', 'drug', 'base', '1979', 'doubl', 'album', 'wall', 'pink', 'floyd', 'film', 'begin', 'pink', 'youth', 'crush', 'love', 'mother', 'sever', 'year', 'later', 'punish', 'teacher', 'school', 'start', 'write', 'poem', 'slowli', 'begin', 'build', 'wall', 'around', 'protect', 'world', 'outsid', 'film', 'show', 'massiv', 'epic', 'pictur', 'end', 'tear', 'wall', 'break', 'free'], ['sherlock', 'holm', 'watson', 'meet', 'boy', 'english', 'board', 'school', 'holm', 'known', 'deduct', 'abil', 'even', 'youth', 'amaz', 'classmat', 'abil', 'discov', 'plot', 'murder', 'seri', 'british', 'busi', 'men', 'egyptian', 'cult', 'move', 'stop'], ['littl', 'boy', 'name', 'andi', 'love', 'room', 'play', 'toy', 'especi', 'doll', 'name', 'woodi', 'toy', 'andi', 'come', 'life', 'woodi', 'believ', 'life', 'toy', 'good', 'howev', 'must', 'worri', 'andi', 'famili', 'move', 'woodi', 'know', 'andi', 'birthday', 'parti', 'woodi', 'realiz', 'andi', 'mother', 'gave', 'action', 'figur', 'known', 'buzz', 'lightyear', 'believ', 'toy', 'quick', 'becom', 'andi', 'new', 'favorit', 'toy', 'woodi', 'consum', 'jealousi', 'tri', 'get', 'rid', 'buzz', 'woodi', 'buzz', 'lost', 'must', 'find', 'way', 'get', 'back', 'andi', 'move', 'without', 'pass', 'ruthless', 'toy', 'killer', 'sid', 'phillip'], ['four', 'jack', 'lad', 'find', 'heavili', 'serious', 'heavili', 'debt', 'east', 'end', 'hard', 'man', 'enforc', 'crook', 'card', 'game', 'overhear', 'neighbour', 'next', 'flat', 'plot', 'hold', 'group', 'depth', 'drug', 'grower', 'hero', 'decid', 'stitch', 'robber', 'turn', 'way', 'confus', 'realli', 'start', 'pair', 'antiqu', 'doubl', 'barrel', 'shotgun', 'miss', 'complet', 'differ', 'scam'], ['922', 'arab', 'courtier', 'ahmad', 'ibn', 'fadlan', 'accompani', 'parti', 'vike', 'barbar', 'north', 'ibn', 'fadlan', 'appal', 'vike', 'custom', 'wanton', 'sexual', 'disregard', 'cleanli', 'cold', 'blood', 'human', 'sacrific', 'learn', 'horrifi', 'truth', 'enlist', 'combat', 'terror', 'slaughter', 'vike', 'devour', 'flesh'], ['villag', 'attack', 'evil', 'ruler', 'snake', 'cult', 'thulsa', 'doom', 'jame', 'earl', 'jone', 'evil', 'warrior', 'thulsa', 'doom', 'warrior', 'kill', 'parent', 'young', 'boy', 'name', 'conan', 'jorg', 'sanz', 'enslav', 'year', 'later', 'conan', 'grow', 'becom', 'mighti', 'warrior', 'train', 'fighter', 'year', 'slave', 'gladiat', 'conan', 'set', 'free', 'conan', 'set', 'quest', 'vow', 'aveng', 'parent', 'solv', 'riddl', 'steel', 'join', 'archer', 'name', 'subotai', 'gerri', 'lopez', 'beauti', 'thief', 'fall', 'love', 'conan', 'valeria', 'sandahl', 'bergman', 'chines', 'wizard', 'mako', 'conan', 'companion', 'set', 'rescu', 'princess', 'yasmina', 'valeri', 'quennessen', 'daughter', 'king', 'osric', 'max', 'von', 'sydow', 'snake', 'cult', 'get', 'reveng', 'thulsa', 'doom', 'aveng', 'parent'], ['oskar', 'schindler', 'vainglori', 'greedi', 'german', 'businessman', 'becom', 'unlik', 'humanitarian', 'amid', 'barbar', 'german', 'nazi', 'reign', 'feel', 'compel', 'turn', 'factori', 'refug', 'jew', 'base', 'true', 'stori', 'oskar', 'schindler', 'manag', 'save', '1100', 'jew', 'gass', 'auschwitz', 'concentr', 'camp', 'testament', 'good'], ['cameron', 'poe', 'high', 'decor', 'unit', 'state', 'armi', 'ranger', 'came', 'home', 'alabama', 'wife', 'tricia', 'run', 'drunken', 'regular', 'tricia', 'work', 'cameron', 'unknow', 'kill', 'one', 'drunk', 'sent', 'feder', 'penitentiari', 'involuntari', 'manslaught', 'seven', 'year', 'cameron', 'becom', 'elig', 'parol', 'home', 'wife', 'daughter', 'unfortun', 'cameron', 'share', 'prison', 'airplan', 'countri', 'danger', 'crimin', 'took', 'control', 'plane', 'plan', 'escap', 'countri', 'cameron', 'find', 'way', 'stop', 'play', 'along', 'meanwhil', 'unit', 'state', 'marshal', 'vincent', 'larkin', 'tri', 'help', 'cameron', 'get', 'free', 'stop', 'crimin', 'includ', 'cyrus', 'virus', 'grissom'], ['adapt', 'rudyard', 'kipl', 'classic', 'tale', 'mowg', 'jungl', 'boy', 'rais', 'wolv', 'lost', 'tiger', 'attack', 'encamp', 'kill', 'father', 'year', 'later', 'find', 'unit', 'childhood', 'love', 'kitti', 'back', 'civil', 'coloni', 'india', 'find', 'far', 'less', 'civil', 'jungl', 'haunt', 'search', 'lost', 'treasur', 'show', 'truli', 'civil', 'member', 'societi'], ['world', 'astound', 'willi', 'wonka', 'year', 'reclus', 'factori', 'announc', 'five', 'lucki', 'peopl', 'given', 'tour', 'factori', 'shown', 'secret', 'amaz', 'candi', 'one', 'win', 'lifetim', 'suppli', 'wonka', 'chocol', 'nobodi', 'want', 'prize', 'young', 'charli', 'famili', 'poor', 'buy', 'even', 'one', 'bar', 'chocol', 'treat', 'buy', 'enough', 'bar', 'find', 'one', 'five', 'golden', 'ticket', 'unlik', 'extrem', 'movieland', 'magic', 'happen', 'charli', 'along', 'four', 'somewhat', 'odious', 'children', 'get', 'chanc', 'lifetim', 'tour', 'factori', 'along', 'way', 'mild', 'disast', 'befal', 'odious', 'children', 'charli', 'beat', 'odd', 'grab', 'brass', 'ring'], ['year', '1936', 'archeolog', 'professor', 'name', 'indiana', 'jone', 'ventur', 'jungl', 'south', 'america', 'search', 'golden', 'statu', 'unfortun', 'set', 'dead', 'trap', 'miracul', 'escap', 'jone', 'hear', 'museum', 'curat', 'name', 'marcus', 'brodi', 'biblic', 'artifact', 'call', 'ark', 'coven', 'hold', 'key', 'human', 'exist', 'jone', 'ventur', 'vast', 'place', 'nepal', 'egypt', 'find', 'artifact', 'howev', 'fight', 'enemi', 'rene', 'belloq', 'band', 'nazi', 'order', 'reach'], ['adam', 'barbara', 'normal', 'coupl', '...', 'happen', 'dead', 'given', 'precious', 'time', 'decor', 'hous', 'make', 'unfortun', 'famili', 'move', 'quiet', 'adam', 'barbara', 'tri', 'scare', 'end', 'becom', 'main', 'attract', 'money', 'make', 'famili', 'call', 'upon', 'beetlejuic', 'help', 'beetlejuic', 'mind', 'help'], ['young', 'boy', 'david', 'moscow', 'make', 'wish', 'carniv', 'machin', 'big', 'wake', 'follow', 'morn', 'find', 'grant', 'bodi', 'grown', 'older', 'overnight', 'still', 'year', 'old', 'boy', 'insid', 'must', 'learn', 'cope', 'unfamiliar', 'world', 'grown', 'up', 'includ', 'get', 'job', 'first', 'romant', 'encount', 'woman', 'find', 'strang', 'world'], ['cultur', 'clash', 'temper', 'flare', 'two', 'cop', 'name', 'detect', 'inspector', 'lee', 'hong', 'kong', 'detect', 'detect', 'jame', 'carter', 'fbi', 'big', 'mouth', 'work', 'alon', 'los', 'angel', 'cop', 'differ', 'world', 'discov', 'one', 'thing', 'common', 'stand', 'time', 'run', 'must', 'join', 'forc', 'catch', 'crimin', 'save', 'eleven', 'year', 'old', 'chines', 'girl', 'chines', 'consul', 'name', 'soo', 'yung'], ['thing', 'seem', 'chang', 'much', 'wabasha', 'counti', 'max', 'john', 'still', 'fight', 'year', 'grandpa', 'still', 'drink', 'smoke', 'chase', 'women', 'nobodi', 'abl', 'catch', 'fabl', 'catfish', 'hunter', 'gigant', 'catfish', 'actual', 'smile', 'fishermen', 'tri', 'snare', 'six', 'month', 'ago', 'john', 'marri', 'new', 'girl', 'town', 'ariel', 'peopl', 'begin', 'suspect', 'max', 'might', 'miss', 'someth', 'similar', 'life', 'joy', 'max', 'claim', 'left', 'life', 'fish', 'might', 'chang', 'new', 'owner', 'bait', 'shop'], ['film', 'two', 'homicid', 'detect', 'morgan', 'freeman', 'brad', 'pitt', 'desper', 'hunt', 'serial', 'killer', 'justifi', 'crime', 'absolut', 'world', 'ignor', 'seven', 'dead', 'sin', 'movi', 'take', 'tortur', 'remain', 'one', 'victim', 'next', 'sociopath', 'john', 'doe', 'kevin', 'spacey', 'sermon', 'detect', 'somerset', 'mill', 'one', 'sin', 'time', 'sin', 'gluttoni', 'come', 'first', 'murder', 'terribl', 'capac', 'graphic', 'demonstr', 'dark', 'subdu', 'tone', 'characterist', 'film', 'noir', 'season', 'cultur', 'jade', 'somerset', 'research', 'seven', 'dead', 'sin', 'effort', 'understand', 'killer', 'modus', 'operandi', 'bright', 'green', 'impuls', 'detect', 'mill', 'pitt', 'scoff', 'effort', 'get', 'insid', 'mind', 'killer', '...'], ['take', 'place', 'dystopian', 'australia', 'near', 'futur', 'mad', 'max', 'tell', 'stori', 'highway', 'patrolman', 'cruis', 'squalid', 'back', 'road', 'becom', 'breed', 'ground', 'crimin', 'forag', 'gasolin', 'scrap', 'wife', 'child', 'meet', 'grisli', 'end', 'hand', 'motorcycl', 'gang', 'max', 'set', 'across', 'barren', 'wasteland', 'search', 'reveng'], ['marion', 'crane', 'steal', 'lot', 'cash', 'man', 'boss', 'busi', 'way', 'see', 'boyfriend', 'stop', 'old', 'motel', 'run', 'odd', 'norman', 'bate', 'murder', 'shower', 'sister', 'boyfriend', 'privat', 'investig', 'tri', 'find', 'learn', 'norman', 'bate'], ['weatherman', 'bill', 'hard', 'tri', 'get', 'tornado', 'hunter', 'wife', 'sign', 'divorc', 'paper', 'marri', 'girlfriend', 'melissa', 'mother', 'natur', 'form', 'seri', 'intens', 'storm', 'sweep', 'across', 'oklahoma', 'plan', 'soon', 'three', 'join', 'team', 'stormchas', 'attempt', 'insert', 'revolutionari', 'measur', 'devic', 'heart', 'sever', 'extrem', 'violent', 'tornado'], ['john', 'dunbar', 'dub', 'hero', 'accident', 'lead', 'union', 'troop', 'victori', 'civil', 'war', 'request', 'posit', 'western', 'frontier', 'find', 'desert', 'soon', 'find', 'alon', 'meet', 'wolf', 'dub', 'two', 'sock', 'curious', 'indian', 'tribe', 'dunbar', 'quick', 'make', 'friend', 'tribe', 'discov', 'white', 'woman', 'rais', 'indian', 'gradual', 'earn', 'respect', 'nativ', 'peopl', 'shed', 'white', 'man', 'way'], ['year', 'old', 'alan', 'abernathi', 'son', 'toy', 'store', 'owner', 'tri', 'new', 'action', 'figur', 'commando', 'elit', 'gorgonit', 'three', 'month', 'ago', 'toy', 'compani', 'believ', 'onto', 'someth', 'employ', 'latest', 'govern', 'militari', 'technolog', 'seri', 'action', 'figur', 'enabl', 'talk', 'underestim', 'power', 'special', 'micro', 'chip', 'employ', 'howev', 'two', 'oppos', 'side', 'toy', 'line', 'start', 'think', 'engag', 'real', 'combat', 'commando', 'elit', 'vow', 'wipe', 'gorgonit', 'suburban', 'neighborhood', 'alan', 'neighbor', 'christi', 'fimpl', 'massiv', 'crush', 'help', 'kind', 'gorgonit', 'must', 'protect', 'home', 'famili', 'commando'], ['swackhamm', 'owner', 'amus', 'park', 'planet', 'moron', 'mountain', 'desper', 'get', 'new', 'attract', 'decid', 'looney', 'tune', 'charact', 'would', 'perfect', 'send', 'diminut', 'underl', 'get', 'whether', 'bug', 'bunni', 'want', 'well', 'arm', 'size', 'bug', 'bunni', 'forc', 'trick', 'agre', 'competit', 'determin', 'freedom', 'take', 'advantag', 'puni', 'stubbi', 'leg', 'foe', 'gang', 'select', 'basketbal', 'surest', 'chanc', 'win', 'howev', 'nerdluck', 'turn', 'tabl', 'steal', 'talent', 'lead', 'profession', 'basketbal', 'star', 'becom', 'massiv', 'basketbal', 'bruiser', 'known', 'monstar', 'desper', 'bug', 'bunni', 'call', 'aid', 'micheal', 'jordan', 'babe', 'ruth', 'basketbal', 'help', 'chanc', 'win', 'freedom'], ['eccentr', 'toymak', 'last', 'wish', 'brother', 'take', 'run', 'busi', 'brother', 'militari', 'general', 'touch', 'toymak', 'touch', 'realiti', 'busi', 'realli', 'given', 'lesli', 'much', 'like', 'toymak', 'father', 'general', 'start', 'make', 'weapon', 'instead', 'toy', 'lesli', 'decid', 'take', 'action'], ['scientif', 'experi', 'unknow', 'bring', 'extraterrestri', 'life', 'form', 'earth', 'laser', 'beam', 'first', 'cigar', 'smoke', 'drake', 'howard', 'duck', 'planet', 'kid', 'tri', 'keep', 'greedi', 'scientist', 'help', 'back', 'planet', 'much', 'less', 'friend', 'arriv', 'beam', '...'], ['sewer', 'gotham', 'citi', 'rooftop', 'gotham', 'citi', 'penguin', 'want', 'know', 'came', 'well', 'villain', 'way', 'catwoman', 'plan', 'kill', 'rich', 'man', 'gotham', 'max', 'shreak', 'battl', 'millionair', 'bruce', 'wayn', 'ladi', 'men', 'secret', 'bruce', 'wayn', 'back', 'bat', 'man', 'tri', 'stop', 'penguin', 'max', 'help', 'penguin', 'steal', 'gotham', 'citi', 'selina', 'kyle', 'catwoman', 'tri', 'help', 'penguin', 'know', 'man', 'murder', 'target', 'also', 'murder', 'help', 'four', 'men', 'goal', 'take', 'gotham', 'crime', 'win', 'gotham', 'citi', 'assassin', 'two', 'men', 'money', 'gotham', 'citi', 'number', 'one', 'rich', 'man'], ['1412', 'young', 'girl', 'call', 'jeann', 'born', 'domrémi', 'franc', 'time', 'hard', 'hunderd', 'year', 'war', 'england', 'go', 'sinc', '1337', 'english', 'knight', 'soldier', 'roam', 'countri', 'jeann', 'develop', 'religi', 'young', 'woman', 'confess', 'sever', 'time', 'day', 'age', 'first', 'vision', 'find', 'sword', 'come', 'home', 'find', 'english', 'level', 'home', 'town', 'year', '1428', 'know', 'mission', 'rid', 'franc', 'english', 'set', 'meet', 'charl', 'dauphin', 'desper', 'militari', 'situat', 'welcom', 'help', 'give', 'maiden', 'chanc', 'prove', 'divin', 'mission', 'success', 'liber', 'orléan', 'reim', 'dauphin', 'crown', 'tradit', 'cathedr', 'reim', 'need', 'anymor', 'sinc', 'wish', 'satisfi', 'jeann', 'arc', 'get', 'set', 'trap', 'imprison', 'burgundian', 'trial', 'english', 'law', 'forc', 'tell', 'divin', 'vision', 'continu', 'sinc', 'childhood', 'condemn', 'witchcraft', 'consid', 'relaps', 'heret', 'sentenc', 'death', 'jeann', 'arc', 'burnt', 'aliv', 'marketplac', 'rouen', 'may', '30th', '1431', 'year', 'age'], ['abort', 'clinic', 'worker', 'special', 'heritag', 'enlist', 'prevent', 'two', 'angel', 'reenter', 'heaven', 'thus', 'undo', 'fabric', 'univers', 'along', 'way', 'aid', 'two', 'prophet', 'jay', 'silent', 'bob', 'help', 'rufus', '13th', 'apostl', 'must', 'stop', 'stand', 'way', 'prevent', 'angel', 'enter', 'heaven'], ['gulliv', 'wash', 'ashor', 'lilliput', 'attempt', 'prevent', 'war', 'tini', 'kingdom', 'equal', 'minuscul', 'rival', 'blefuscu', 'well', 'smooth', 'way', 'romanc', 'princess', 'princ', 'oppos', 'land', 'altern', 'aid', 'hamper', 'lilliputian', 'town', 'crier', 'general', 'fussbudget', 'gabbi', 'life', 'threaten', 'situat', 'develop', 'bumbl', 'trio', 'blefuscu', 'spi', 'sneak', 'snoop', 'snitch', 'manag', 'steal', 'gulliv', 'pistol'], ['wayn', 'still', 'live', 'home', 'world', 'class', 'collect', 'name', 'tag', 'job', 'tri', 'public', 'access', 'show', 'local', 'station', 'decid', 'hire', 'sidekick', 'garth', 'show', 'profession', 'wayn', 'garth', 'find', 'longer', 'wayn', 'fall', 'bass', 'guitarist', 'use', 'garth', 'video', 'contact', 'help', 'career', 'along', 'know', 'ben', 'oliv', 'sleazi', 'advertis', 'guy', 'ruin', 'show', 'probabl', 'take', 'away', 'fail'], ['peter', 'pan', 'william', 'grown', 'cut', 'throat', 'merger', 'acquisit', 'lawyer', 'marri', 'wendi', 'granddaught', 'captain', 'hook', 'hoffman', 'kidnap', 'children', 'peter', 'return', 'never', 'land', 'tinkerbel', 'robert', 'help', 'lost', 'boy', 'must', 'rememb', 'peter', 'pan', 'order', 'save', 'children', 'battl', 'captain', 'hook'], ['truck', 'driver', 'jack', 'burton', 'arriv', 'chinatown', 'san', 'francisco', 'goe', 'airport', 'chines', 'friend', 'wang', 'chi', 'welcom', 'green', 'eye', 'fiancé', 'miao', 'yin', 'arriv', 'china', 'howev', 'kidnap', 'arriv', 'chines', 'street', 'gang', 'jack', 'wang', 'chase', 'group', 'soon', 'learn', 'power', 'evil', 'sorcer', 'call', 'david', 'pan', 'curs', 'two', 'thousand', 'year', 'ago', 'exist', 'without', 'physic', 'bodi', 'need', 'marri', 'woman', 'green', 'eye', 'retriev', 'physic', 'bodi', 'miao', 'chosen', 'one', 'jack', 'wang', 'team', 'lawyer', 'graci', 'law', 'bus', 'driver', 'sorcer', 'apprentic', 'egg', 'shen', 'friend', 'embark', 'great', 'adventur', 'underground', 'chinatown', 'face', 'world', 'magician', 'magic', 'monster', 'martial', 'art', 'fighter'], ['jule', 'winnfield', 'samuel', 'jackson', 'vincent', 'vega', 'john', 'travolta', 'two', 'hit', 'men', 'retriev', 'suitcas', 'stolen', 'employ', 'mob', 'boss', 'marsellus', 'wallac', 'ving', 'rhame', 'wallac', 'also', 'ask', 'vincent', 'take', 'wife', 'mia', 'uma', 'thurman', 'day', 'later', 'wallac', 'town', 'butch', 'coolidg', 'bruce', 'willi', 'age', 'boxer', 'paid', 'wallac', 'lose', 'fight', 'live', 'seem', 'unrel', 'peopl', 'woven', 'togeth', 'compris', 'seri', 'funni', 'bizarr', 'uncal', 'incid'], ['highland', 'scotland', '1700s', 'rob', 'roy', 'tri', 'lead', 'small', 'town', 'better', 'futur', 'borrow', 'money', 'local', 'nobil', 'buy', 'cattl', 'herd', 'market', 'money', 'stolen', 'rob', 'forc', 'robin', 'hood', 'lifestyl', 'defend', 'famili', 'honour'], ['martin', 'rigg', 'cop', 'suicid', 'tendenc', 'roger', 'murtaugh', 'unlucki', 'polic', 'offic', 'rigg', 'assign', 'togeth', 'uncov', 'huge', 'drug', 'smuggl', 'oper', 'success', 'rate', 'grow', 'friendship'], ['jame', 'happi', 'life', 'english', 'seasid', 'rude', 'end', 'parent', 'kill', 'rhinocero', 'goe', 'live', 'two', 'horrid', 'aunt', 'dare', 'save', 'life', 'spider', 'come', 'possess', 'magic', 'boil', 'crocodil', 'tongu', 'enorm', 'peach', 'start', 'grow', 'garden', 'ventur', 'insid', 'meet', 'spider', 'number', 'new', 'friend', 'includ', 'ladybug', 'centiped', 'help', 'plan', 'tri', 'get', 'new', 'york'], ['juli', 'back', 'colleg', 'new', 'friend', 'win', 'weekend', 'trip', 'island', 'way', 'someon', 'die', 'girl', 'torment', 'island'], ['group', 'report', 'tri', 'deciph', 'last', 'word', 'ever', 'spoken', 'charl', 'foster', 'kane', 'millionair', 'newspap', 'tycoon', 'rosebud', 'film', 'begin', 'news', 'reel', 'detail', 'kane', 'life', 'mass', 'shown', 'flashback', 'kane', 'life', 'report', 'investig', 'viewer', 'see', 'display', 'fascin', 'man', 'rise', 'fame', 'eventu', 'fell', 'top', 'world'], ['film', 'open', 'tuxedo', 'clad', 'men', 'show', 'penitentiari', 'meet', 'friend', 'releas', 'three', 'year', 'prison', 'go', 'straight', 'jail', 'marri', 'girl', 'friend', 'rout', 'wed', 'one', 'men', 'ask', 'stop', 'bank', 'pick', 'cash', 'turn', 'want', 'bank', 'robber', 'use', 'shakespear', 'passag', 'robberi', 'thus', 'becom', 'known', 'hamlet', 'soon', 'five', 'men', 'caught', 'bank', 'involv', 'robberi', 'end', 'hostag', 'situat', 'hostag', 'negoti', 'show', 'turn', 'hamlet', 'father', 'men', 'known', 'rob', 'quirki', 'comedi', 'take', 'left', 'turn', 'hostag', 'work', 'support', 'men'], ['former', 'australian', 'policeman', 'live', 'post', 'apocalypt', 'australian', 'outback', 'warrior', 'agre', 'help', 'communiti', 'survivor', 'live', 'gasolin', 'refineri', 'defend', 'gasolin', 'suppli', 'evil', 'barbarian', 'warrior'], ['english', 'languag', 'film', 'made', 'america', 'adapt', 'novel', 'german', 'author', 'erich', 'maria', 'remarqu', 'film', 'follow', 'group', 'german', 'schoolboy', 'talk', 'enlist', 'begin', 'world', 'war', 'jingoist', 'teacher', 'stori', 'told', 'entir', 'experi', 'young', 'german', 'recruit', 'highlight', 'tragedi', 'war', 'eye', 'individu', 'boy', 'wit', 'death', 'mutil', 'around', 'preconcept', 'enemi', 'right', 'wrong', 'conflict', 'disappear', 'leav', 'angri', 'bewild', 'highlight', 'scene', 'paul', 'mortal', 'wound', 'french', 'soldier', 'weep', 'bitter', 'fight', 'save', 'life', 'trap', 'shell', 'crater', 'bodi', 'film', 'heroism', 'drudgeri', 'futil', 'gulf', 'concept', 'war', 'actual'], ['sever', 'british', 'agent', 'murder', 'jame', 'bond', 'sent', 'new', 'orlean', 'investig', 'mysteri', 'death', 'big', 'come', 'knowledg', 'self', 'produc', 'heroin', 'along', 'journey', 'meet', 'tee', 'hee', 'claw', 'hand', 'baron', 'samedi', 'voodoo', 'master', 'solitair', 'tarot', 'card', 'reader', 'bond', 'must', 'travel', 'new', 'orlean', 'deep', 'bayou'], ['commerci', 'crew', 'aboard', 'deep', 'space', 'tow', 'vessel', 'nostromo', 'way', 'home', 'pick', 'sos', 'warn', 'distant', 'moon', 'know', 'sos', 'warn', 'like', 'ordinari', 'warn', 'call', 'pick', 'signal', 'crew', 'realiz', 'alon', 'spaceship', 'alien', 'stowaway', 'cargo', 'ship'], ['young', 'hobbit', 'name', 'frodo', 'guard', 'thrown', 'amaz', 'adventur', 'appoint', 'job', 'destroy', 'one', 'ring', 'creat', 'dark', 'lord', 'sauron', 'assign', 'warrior', 'includ', 'gandalf', 'squir', 'aragorn', 'hurt', 'boromir', 'cox', 'go', 'easi', 'journey', 'fellowship', 'ring', 'ultim', 'quest', 'rid', 'middl', 'earth', 'evil'], ['mrs', 'brisbi', 'widow', 'mous', 'live', 'cinder', 'block', 'children', 'fitzgibbon', 'farm', 'prepar', 'move', 'famili', 'field', 'live', 'plow', 'time', 'approach', 'howev', 'son', 'timothi', 'fallen', 'ill', 'move', 'could', 'prove', 'fatal', 'mrs', 'brisbi', 'visit', 'great', 'owl', 'wise', 'creatur', 'advis', 'visit', 'mysteri', 'group', 'rat', 'live', 'beneath', 'rose', 'bush', 'farm', 'upon', 'visit', 'rat', 'brisbi', 'meet', 'nicodemus', 'wise', 'mystic', 'leader', 'rat', 'justin', 'friend', 'rat', 'immedi', 'becom', 'attach', 'mrs', 'brisbi', 'learn', 'late', 'husband', 'jonathon', 'brisbi', 'along', 'rat', 'part', 'seri', 'experi', 'place', 'known', 'reveal', 'earlier', 'stori', 'nation', 'institut', 'mental', 'health', 'experi', 'perform', 'mice', 'rat', 'boost', 'intellig', 'allow', 'read', 'without', 'taught', 'understand', 'thing', 'complex', 'mechan', 'electr', 'rat', 'brisbi', 'escap', 'came', 'live', 'fitzgibbon', 'farm', 'rat', 'creat', 'home', 'mrs', 'fitzgibbon', 'rose', 'bush', 'creat', 'elabor', 'habit', 'beauti', 'chamber', 'elev', 'christma', 'light', 'howev', 'rat', 'unhappi', 'depend', 'human', 'steal', 'electr', 'concoct', 'plan', 'leav', 'farm', 'live', 'independ', 'husband', 'prior', 'relationship', 'rat', 'agre', 'help', 'mrs', 'brisbi', 'move', 'home', 'path', 'plow', 'howev', 'evil', 'jenner', 'unwil', 'accomplic', 'sullivan', 'wish', 'remain', 'beneath', 'rose', 'bush', 'yet', 'plot', 'kill', 'nicodemus', 'move'], ['carl', 'denham', 'need', 'finish', 'movi', 'perfect', 'locat', 'skull', 'island', 'still', 'need', 'find', 'lead', 'ladi', 'soon', 'unfortun', 'soul', 'ann', 'darrow', 'one', 'know', 'encount', 'island', 'mysteri', 'reach', 'soon', 'find', 'live', 'hidden', 'island', 'giant', 'gorilla', 'beast', 'ann', 'grasp', 'carl', 'ann', 'new', 'love', 'jack', 'driscol', 'must', 'travel', 'jungl', 'look', 'kong', 'ann', 'whilst', 'avoid', 'sort', 'creatur', 'beast'], ['toon', 'star', 'roger', 'worri', 'wife', 'jessica', 'play', 'pattycak', 'someon', 'els', 'studio', 'hire', 'detect', 'eddi', 'valiant', 'snoop', 'stake', 'quick', 'rais', 'marvin', 'acm', 'found', 'dead', 'roger', 'prime', 'suspect', 'groundbreak', 'interact', 'live', 'anim', 'charact', 'lot', 'refer', 'classic', 'anim'], ['2274', 'surfac', 'seem', 'idyl', 'societi', 'live', 'citi', 'within', 'enclos', 'dome', 'littl', 'work', 'human', 'perform', 'inhabit', 'free', 'pursu', 'pleasur', 'life', 'one', 'catch', 'howev', 'life', 'limit', 'reach', 'termin', 'quasi', 'religi', 'ceremoni', 'known', 'carousel', 'known', 'runner', 'tri', 'escap', 'fate', 'time', 'come', 'job', 'sandmen', 'track', 'kill', 'logan', 'man', 'sever', 'year', 'termin', 'date', 'think', 'noth', 'job', 'soon', 'meet', 'young', 'woman', 'jessica', 'order', 'becom', 'runner', 'infiltr', 'communiti', 'outsid', 'dome', 'known', 'sanctuari', 'destroy', 'pursu', 'friend', 'franci', 'also', 'sandman', 'logan', 'jessica', 'find', 'way', 'outsid', 'discov', 'beauti', 'virtual', 'uninhabit', 'world', 'logan', 'realiz', 'must', 'return', 'dome', 'tell', 'found', 'set', 'free'], ['bottl', 'contain', 'plea', 'help', 'littl', 'girl', 'name', 'penni', 'make', 'way', 'rescu', 'aid', 'societi', 'mous', 'organ', 'basement', 'unit', 'nation', 'build', 'dedic', 'rescu', 'well', 'anyon', 'need', 'brave', 'mous', 'miss', 'bianca', 'chosen', 'partner', 'shi', 'janitor', 'bernard', 'rescu', 'girl', 'search', 'clue', 'penni', 'home', 'morningsid', 'orphanag', 'new', 'york', 'citi', 'two', 'mice', 'discov', 'girl', 'kidnap', 'evil', 'pawn', 'shop', 'owner', 'madam', 'medusa', 'companion', 'snoop', 'back', 'orvill', 'albatross', 'miss', 'bianca', 'bernard', 'travel', 'terrifi', 'gloomi', 'devil', 'bayou', 'learn', 'shock', 'truth', 'innoc', 'young', 'girl', 'forc', 'danger', 'dark', 'underground', 'pirat', 'cave', 'must', 'find', 'devil', 'eye', 'world', 'largest', 'diamond', 'madam', 'medusa', 'greatest', 'obsess', 'return', 'safe', 'home', 'miss', 'bianca', 'bernard', 'penni', 'combat', 'madam', 'medusa', 'two', 'feroci', 'pet', 'allig', 'brutus', 'nero', 'help', 'elli', 'mae', 'evinrud', 'dragonfli', 'well', 'surviv', 'rage', 'tide', 'insid', 'horribl', 'pirat', 'cave'], ['centuri', 'ago', 'land', 'prydain', 'young', 'man', 'name', 'taran', 'given', 'task', 'protect', 'hen', 'wen', 'magic', 'oracular', 'pig', 'know', 'locat', 'mystic', 'black', 'cauldron', 'easi', 'task', 'evil', 'horn', 'king', 'stop', 'noth', 'get', 'cauldron'], ['dougla', 'quaid', 'haunt', 'recur', 'dream', 'journey', 'mar', 'hope', 'find', 'dream', 'buy', 'holiday', 'rekal', 'inc', 'sell', 'implant', 'memori', 'someth', 'goe', 'wrong', 'memori', 'implant', 'rememb', 'secret', 'agent', 'fight', 'evil', 'mar', 'administr', 'cohaagen', 'stori', 'realli', 'begin', 'rollercoast', 'ride', 'massiv', 'end', 'movi'], ['four', 'newton', 'brother', 'poor', 'farmer', 'famili', '1920s', 'oldest', 'willi', 'one', 'day', 'realiz', 'futur', 'field', 'offer', 'brother', 'becom', 'bank', 'robber', 'soon', 'famili', 'agre', 'becom', 'famous', 'robber', 'five', 'year', 'later', 'execut', 'greatest', 'train', 'robberi', 'american', 'histori'], ['bastian', 'young', 'boy', 'live', 'dreari', 'life', 'torment', 'school', 'bulli', 'one', 'occas', 'escap', 'book', 'shop', 'old', 'propriet', 'reveal', 'ancient', 'stori', 'book', 'warn', 'danger', 'short', 'borrow', 'book', 'begin', 'read', 'school', 'attic', 'drawn', 'mythic', 'land', 'fantasia', 'desper', 'need', 'hero', 'save', 'destruct'], ['robert', 'dean', 'mild', 'manner', 'lawyer', 'work', 'washington', 'trail', 'kingpin', 'name', 'pintero', 'meanwhil', 'politician', 'name', 'thoma', 'reynold', 'negoti', 'congressman', 'phillip', 'hammersley', 'new', 'surveil', 'system', 'satellit', 'hammersley', 'declin', 'reynold', 'hammersley', 'kill', 'murder', 'caught', 'tape', 'taper', 'chase', 'reynold', 'team', 'nsa', 'agent', 'guy', 'must', 'ditch', 'tape', 'plant', 'dean', 'unbeknownst', 'dean', 'nsa', 'decid', 'get', 'dean', 'life', 'dean', 'life', 'began', 'fall', 'apart', 'around', 'wife', 'job', 'gone', 'dean', 'want', 'find', 'go', 'meet', 'man', 'name', 'brill', 'tell', 'dean', 'someth', 'govern', 'want', 'dean', 'brill', 'formul', 'plan', 'get', 'dean', 'life', 'back', 'turn', 'tabl', 'reynold'], ['tommi', 'callahan', 'slow', 'wit', 'clumsi', 'guy', 'recent', 'graduat', 'colleg', 'attend', 'seven', 'year', 'father', 'big', 'tom', 'callahan', 'own', 'auto', 'part', 'factori', 'ohio', 'tommi', 'arriv', 'back', 'home', 'find', 'posit', 'factori', 'wait', 'dad', 'also', 'introduc', 'tommi', 'new', 'brake', 'pad', 'divis', 'factori', 'tommi', 'soon', 'stepmoth', 'bever', 'son', 'paul', 'big', 'tom', 'die', 'factori', 'threaten', 'unless', 'new', 'brake', 'pad', 'sold', 'therefor', 'tommi', 'must', 'road', 'sell', 'along', 'assist', 'richard', 'big', 'tom', 'right', 'hand', 'man', 'tommi', 'save', 'compani', 'factori', 'town'], ['arthur', 'aka', 'wart', 'young', 'boy', 'aspir', 'knight', 'squir', 'hunt', 'trip', 'fall', 'merlin', 'power', 'amnesiac', 'wizard', 'plan', 'wart', 'beyond', 'mere', 'squiredom', 'start', 'tri', 'give', 'wart', 'educ', 'whatev', 'believ', 'one', 'educ', 'one', 'anywher', 'needless', 'say', 'quit', 'work', 'way'], ['sister', 'act', 'reno', 'loung', 'singer', 'name', 'delori', 'van', 'carter', 'wit', 'mobster', 'boyfriend', 'kill', 'employe', 'hidden', 'convent', 'wit', 'protect', 'program', 'soon', 'make', 'friend', 'nun', 'especi', 'sister', 'mari', 'robert', 'sister', 'mari', 'lazura', 'sister', 'mari', 'patrick', 'mother', 'superior', 'catch', 'delori', 'go', 'bar', 'night', 'follow', 'mari', 'robert', 'mari', 'patrick', 'order', 'join', 'church', 'choir', 'find', 'coach', 'choir', 'turn', 'swingin', 'singin', 'sister', 'choir', 'prove', 'big', 'success', 'surround', 'neighborhood', 'delori', 'boyfriend', 'track', '...'], ['hacker', 'arcad', 'owner', 'kevin', 'flynn', 'digit', 'broken', 'data', 'stream', 'villain', 'softwar', 'pirat', 'known', 'master', 'control', 'reconstitut', 'intern', 'graphic', 'world', 'comput', 'ultim', 'blaze', 'color', 'geometr', 'intens', 'landscap', 'cyberspac', 'flynn', 'join', 'forc', 'tron', 'outmaneuv', 'master', 'control', 'program', 'hold', 'captiv', 'equival', 'gigant', 'infinit', 'challeng', 'comput', 'game'], ['peter', 'venkman', 'ray', 'stantz', 'egon', 'work', 'univers', 'delv', 'paranorm', 'fiddl', 'mani', 'uneth', 'experi', 'student', 'kick', 'univers', 'realli', 'understand', 'knowledg', 'paranorm', 'busi', 'new', 'snazzi', 'busi', 'name', 'ghostbust', 'live', 'old', 'firehous', 'build', 'work', 'call', 'rid', 'new', 'york', 'citi', 'paranorm', 'phenomenon', 'everyon', 'whim', '....', 'price', 'make', 'nation', 'press', 'media', 'think', 'pressur', 'everybodi', 'ghostbust', 'caus', 'thrown', 'jail', 'epa', 'mayor', 'take', 'chanc', 'call', 'help', 'save', 'citi', 'unbeknownst', 'long', 'dead', 'gozer', 'worship', 'evo', 'shandor', 'erect', 'downtown', 'apart', 'build', 'caus', 'paranorm', 'activ', 'find', 'build', 'could', 'resurrect', 'ancient', 'hittit', 'god', 'gozer', 'bring', 'end', 'human', 'gonna', 'call', 'stop', 'terribl', 'world', 'end', 'menac'], ['myth', 'king', 'arthur', 'brought', 'screen', 'uthur', 'pendragon', 'given', 'mystic', 'sword', 'excalibur', 'wizard', 'merlin', 'death', 'uthur', 'buri', 'sword', 'stone', 'next', 'man', 'pull', 'king', 'england', 'year', 'later', 'arthur', 'uthur', 'bastard', 'son', 'draw', 'excalibur', 'becom', 'king', 'guid', 'merlin', 'arthur', 'marri', 'gueniver', 'gather', 'knight', 'round', 'tabl', 'arthur', 'evil', 'half', 'sister', 'morgana', 'sire', 'son', 'may', 'prove', 'downfal'], ['derek', 'vineyard', 'parol', 'serv', 'year', 'prison', 'brutal', 'kill', 'two', 'black', 'men', 'tri', 'break', 'steal', 'truck', 'brother', 'danni', 'vineyard', 'narrat', 'learn', 'go', 'prison', 'derek', 'skinhead', 'leader', 'violent', 'white', 'supremacist', 'gang', 'commit', 'act', 'racial', 'crime', 'throughout', 'action', 'great', 'influenc', 'danni', 'reform', 'fresh', 'prison', 'derek', 'sever', 'contact', 'gang', 'becom', 'determin', 'keep', 'danni', 'go', 'violent', 'path'], ['detroit', 'futur', 'crime', 'ridden', 'run', 'massiv', 'compani', 'compani', 'develop', 'huge', 'crime', 'fight', 'robot', 'unfortun', 'develop', 'rather', 'danger', 'glitch', 'compani', 'see', 'way', 'get', 'back', 'favor', 'public', 'policeman', 'alex', 'murphi', 'kill', 'street', 'gang', 'murphi', 'bodi', 'reconstruct', 'within', 'steel', 'shell', 'call', 'robocop', 'robocop', 'success', 'crimin', 'becom', 'target', 'supervillian', 'boddick'], ['charli', 'poet', 'much', 'luck', 'women', 'meet', 'harriet', 'girl', 'dream', 'nightmar', 'charli', 'begin', 'suspect', 'harriet', 'woman', 'marri', 'kill', 'husband'], ['fievel', 'young', 'russian', 'mous', 'separ', 'parent', 'way', 'america', 'land', 'think', 'without', 'cat', 'arriv', 'alon', 'new', 'world', 'keep', 'hope', 'search', 'famili', 'make', 'new', 'friend', 'run', 'dodg', 'cat', 'thought', 'rid'], ['castl', 'high', 'top', 'hill', 'live', 'inventor', 'greatest', 'creation', 'edward', 'near', 'complet', 'person', 'creator', 'die', 'could', 'finish', 'edward', 'hand', 'instead', 'left', 'metal', 'scissor', 'hand', 'sinc', 'live', 'alon', 'kind', 'ladi', 'call', 'peg', 'discov', 'welcom', 'home', 'first', 'everyon', 'welcom', 'communiti', 'soon', 'thing', 'begin', 'take', 'chang', 'wors'], ['sir', 'robin', 'locksley', 'defend', 'downtrodden', 'saxon', 'run', 'afoul', 'norman', 'author', 'forc', 'turn', 'outlaw', 'band', 'merri', 'men', 'rob', 'rich', 'give', 'poor', 'still', 'time', 'woo', 'love', 'maid', 'marian', 'foil', 'cruel', 'sir', 'guy', 'gisbourn', 'keep', 'nefari', 'princ', 'john', 'throne'], ['con', 'artist', 'escap', 'deal', 'gone', 'wrong', 'new', 'york', 'wind', 'aussi', 'outback', 'strang', 'town', 'whose', 'inhabit', 'oddbal', 'collect', 'misfit'], ['alic', 'daydream', 'young', 'girl', 'find', 'learn', 'poem', 'listen', 'literatur', 'bore', 'prefer', 'stori', 'pictur', 'live', 'insid', 'imagin', 'one', 'day', 'endur', 'poetri', 'read', 'spot', 'larg', 'white', 'rabbit', '...', 'dress', 'jacket', 'carri', 'larg', 'watch', 'scurri', 'say', 'late', 'import', 'date', 'follow', 'forest', 'disappear', 'rabbit', 'hole', 'alic', 'follow', 'lead', 'manner', 'discoveri', 'charact', 'adventur'], ['histori', 'turn', 'comic', 'head', '10th', 'centuri', 'england', 'king', 'arthur', 'travel', 'countrysid', 'find', 'knight', 'join', 'round', 'tabl', 'camelot', 'gather', 'men', 'tale', 'bit', 'parti', 'camelot', 'mani', 'decid', 'leav', 'stop', 'god', 'send', 'quest', 'find', 'holi', 'grail', 'seri', 'individu', 'adventur', 'knight', 'reunit', 'must', 'face', 'wizard', 'name', 'tim', 'killer', 'rabbit', 'lesson', 'use', 'holi', 'hand', 'grenad', 'quest', 'come', 'end', 'howev', 'polic', 'interven', 'would', 'expect', 'monti', 'python', 'movi'], ['midst', 'elabor', 'conspiraci', 'expert', 'negoti', 'driven', 'edg', 'frame', 'murder', 'partner', 'well', 'embezzl', 'money', 'depart', 'pension', 'fund', 'chanc', 'prove', 'innoc', 'take', 'hostag', 'acquir', 'servic', 'anoth', 'expert', 'negoti', 'find', 'run', 'conspiraci', 'late'], ['turtl', 'find', 'ooz', 'substanc', 'made', 'mutat', 'came', 'unfortun', 'shredder', 'learn', 'use', 'enhanc', 'turtl', 'prove', 'better', 'ninja', 'fighter'], ['martin', 'bishop', 'head', 'group', 'expert', 'specialis', 'test', 'secur', 'system', 'blackmail', 'govern', 'agent', 'steal', 'top', 'secret', 'black', 'box', 'team', 'find', 'embroil', 'game', 'danger', 'intrigu', 'recov', 'box', 'discov', 'capabl', 'decod', 'exist', 'encrypt', 'system', 'around', 'world', 'agent', 'hire', 'work', 'govern', '...'], ['recount', 'fabl', 'pop', 'rock', 'band', 'form', 'year', 'beatl', 'took', 'america', 'storm', 'earli', '1964', 'jazz', 'aficionado', 'guy', 'patterson', 'unhappili', 'toil', 'famili', 'applianc', 'store', 'recruit', 'band', 'oned', 'later', 'renam', 'wonder', 'regular', 'drummer', 'chad', 'break', 'arm', 'guy', 'inject', 'four', 'four', 'rock', 'beat', 'lead', 'singer', 'jimmi', 'ballad', 'song', 'undeni', 'pop', 'power', 'fling', 'wonder', 'brief', 'whirlwind', 'success', 'tell', 'tale', 'mani', 'american', 'band', 'attempt', 'grab', 'brass', 'ring', 'rock', 'roll', 'wake', 'british', 'invas'], ['theatric', 'movi', 'base', 'televis', 'seri', 'also', 'base', 'popular', 'multiform', 'robot', 'toylin', 'well', 'box', 'offic', 'movi', 'take', 'place', '2005', 'twenti', 'year', 'televis', 'seri', 'chronicl', 'effort', 'heroic', 'autobot', 'defend', 'homeworld', 'cybertron', 'evil', 'decepticon', 'faction', 'seeth', 'anger', 'hatr', 'blind', 'hideous', 'menac', 'head', 'way', 'hideous', 'menac', 'coloss', 'planet', 'known', 'unicron', 'readi', 'consum', 'anyth', 'stand', 'way', 'thing', 'stop', 'unicron', 'autobot', 'matrix', 'leadership', 'possess', 'autobot', 'decepticon', 'unicron', 'order', 'plan', 'take', 'away'], ['jerri', 'work', 'father', 'law', 'car', 'dealership', 'gotten', 'financi', 'problem', 'tri', 'various', 'scheme', 'come', 'money', 'need', 'reason', 'never', 'realli', 'explain', 'assum', 'huge', 'embezzl', 'money', 'dealership', 'discov', 'father', 'law', 'els', 'fall', 'plan', 'set', 'motion', 'earlier', 'two', 'men', 'kidnap', 'wife', 'ransom', 'paid', 'wealthi', 'father', 'seem', 'time', 'day', 'son', 'law', 'moment', 'kidnap', 'thing', 'wrong', 'suppos', 'non', 'violent', 'affair', 'turn', 'bloodi', 'blood', 'ad', 'minut', 'jerri', 'upset', 'bloodsh', 'turn', 'loos', 'pregnant', 'sheriff', 'brainerd', 'tenaci', 'attempt', 'solv', 'three', 'murder', 'jurisdict'], ['two', 'punk', 'live', 'salt', 'lake', 'citi', 'film', 'cover', 'day', 'routin', 'realism', 'charact', 'narrat', 'movi', 'may', 'discuss', 'one', 'punk', 'get', 'ill', 'stay', 'hospit', 'three', 'week', 'come', 'three', 'parti', 'cover', 'one', 'concert', 'includ', 'fight', 'punk', 'redneck', 'other'], ['three', 'best', 'disband', 'musket', 'atho', 'portho', 'arami', 'join', 'young', 'hothead', 'would', 'musket', 'artagnan', 'stop', 'cardin', 'richelieu', 'evil', 'plot', 'form', 'allianc', 'enemi', 'england', 'way', 'mysteri', 'miladi', 'rochefort', 'cardin', 'right', 'hand', 'man', 'announc', 'offici', 'disband', 'king', 'musket', 'three', 'howev', 'refus', 'throw', 'sword', 'atho', 'fighter', 'drinker', 'portho', 'pirat', 'lover', 'arami', 'priest', 'poet', 'arriv', 'pari', 'join', 'musket', 'artagnan', 'uncov', 'cardin', 'plan', 'four', 'set', 'mission', 'protect', 'king', 'countri'], ['john', 'shaft', 'ultim', 'suav', 'black', 'detect', 'first', 'find', 'bumpi', 'leader', 'black', 'crime', 'mob', 'black', 'nation', 'final', 'work', 'white', 'mafia', 'tri', 'blackmail', 'bumpi', 'kidnap', 'daughter'], ['drive', 'home', 'rain', 'fill', 'night', 'straight', 'lace', 'lovebird', 'brad', 'major', 'janet', 'weiss', 'end', 'chanc', 'castl', 'one', 'frank', 'furter', 'strang', 'bizarr', 'entourag', 'find', 'parti', 'ordinari', 'parti', 'ordinari', 'night', 'unveil', 'doctor', 'latest', 'creation', 'rocki', 'horror', 'man', 'made', 'adoni', 'give', 'absolut', 'pleasur', 'cours', 'night', 'frank', 'seduc', 'brad', 'janet', 'janet', 'rocki', 'becom', 'biblic', 'involv', 'everett', 'von', 'scott', 'arriv', 'look', 'nephew', 'eddi', 'frank', 'kill', 'earlier', 'film', 'exceed', 'grand', 'visual', 'music', 'camp', 'satir', 'golden', 'day', 'movi', 'horror', 'scienc', 'fiction', 'genr', 'project', 'along', 'music', 'soundtrack', 'give', 'audienc', 'particip', 'new', 'mean', 'dimens', 'time', 'space', 'shall', 'night', 'brad', 'janet', 'rememb', 'long', 'time', 'sexual', 'kinki', 'rock', 'roll', 'rock', 'opera', 'world', 'gender', 'bend', 'scientist', 'time', 'warp', 'plan'], ['babi', 'girl', 'discov', 'river', 'ranon', 'mim', 'children', 'willow', 'ufgood', 'dwarf', 'farmer', 'magician', 'babi', 'girl', 'taken', 'care', 'willow', 'famili', 'terrifi', 'dog', 'like', 'creatur', 'attack', 'willow', 'villag', 'whilst', 'track', 'babi', 'willow', 'consult', 'villag', 'council', 'wizard', 'high', 'aldwin', 'high', 'aldwin', 'give', 'willow', 'task', 'willow', 'leav', 'villag', 'embark', 'task', 'give', 'babi', 'girl', 'respons', 'person', 'willow', 'soon', 'learn', 'babi', 'elora', 'danan', 'babi', 'girl', 'destin', 'bring', 'downfal', 'evil', 'sorceress', 'queen', 'bavmorda', 'join', 'alli', 'swordsman', 'madmartigan', 'sorceress', 'fin', 'raziel', 'browni', 'franjean', 'rool', 'willow', 'take', 'upon', 'protect', 'elora', 'queen', 'bavmorda', 'intend', 'kill', 'elora', 'prevent', 'elora', 'fulfil', 'destini', 'willow', 'alli', 'pursu', 'queen', 'bavmorda', 'daughter', 'sorsha', 'evil', 'command', 'queen', 'bavmorda', 'armi', 'general', 'kael', 'search', 'elora', 'bring', 'back', 'queen', 'bavmorda', 'castl', 'queen', 'bavmorda', 'bid', 'kill', 'elora', 'ritual', 'prevent', 'propheci', 'downfal'], ['pick', 'superman', 'movi', 'left', 'three', 'crimin', 'general', 'zod', 'terenc', 'stamp', 'ursa', 'sarah', 'dougla', 'non', 'jack', 'halloran', 'planet', 'krypton', 'releas', 'phantom', 'zone', 'nuclear', 'explos', 'space', 'descend', 'upon', 'earth', 'could', 'final', 'rule', 'superman', 'meanwhil', 'love', 'loi', 'lane', 'margot', 'kidder', 'find', 'realli', 'lex', 'luthor', 'gene', 'hackman', 'escap', 'prison', 'determin', 'destroy', 'superman', 'join', 'forc', 'three', 'crimin'], ['stori', 'brian', 'nazareth', 'born', 'day', 'jesus', 'nazareth', 'take', 'differ', 'path', 'life', 'lead', 'conclus', 'brian', 'join', 'polit', 'resist', 'movement', 'aim', 'get', 'roman', 'judea', 'brian', 'score', 'victori', 'sort', 'manag', 'paint', 'polit', 'slogan', 'entir', 'wall', 'citi', 'jerusalem', 'movement', 'effect', 'somehow', 'brian', 'becom', 'prophet', 'gather', 'follow', 'fate', 'seal', 'howev', 'live', 'short', 'life'], ['accid', 'wind', 'road', 'four', 'teen', 'make', 'fatal', 'mistak', 'dump', 'victim', 'bodi', 'sea', 'exact', 'one', 'year', 'later', 'dead', 'man', 'return', 'wateri', 'grave', 'look', 'apolog'], ['releas', 'jake', 'blue', 'prison', 'brother', 'elwood', 'visit', 'penguin', 'last', 'nun', 'rais', 'board', 'school', 'learn', 'archdioces', 'stop', 'support', 'school', 'sell', 'place', 'educ', 'author', 'way', 'keep', 'place', 'open', '5000', 'tax', 'properti', 'paid', 'within', 'day', 'blue', 'brother', 'want', 'help', 'decid', 'put', 'blue', 'band', 'back', 'togeth', 'rais', 'money', 'stage', 'big', 'gig', 'set', 'mission', 'god', 'seem', 'make', 'enemi', 'along', 'way', 'manag', 'come', 'money', 'time'], ['pooh', 'back', 'time', 'gonna', 'blow', 'away', 'almost', 'liter', 'pooh', 'friend', 'gopher', 'warn', 'windsday', 'pooh', 'goe', 'wish', 'piglet', 'happi', 'windsday', 'piglet', 'lift', 'air', 'strong', 'wind', 'pooh', 'grab', 'piglet', 'scarf', 'unravel', 'singl', 'string', 'look', 'fli', 'piglet', 'like', 'kite', 'arriv', 'owl', 'hous', 'wind', 'blow', 'owl', 'inquir', 'whether', 'pooh', 'eeyor', 'agre', 'find', 'new', 'hous', 'owl', 'one', 'night', 'pooh', 'get', 'visit', 'unusu', 'new', 'friend', 'tigger', 'rambuncti', 'tiger', 'like', 'honey', 'hate', 'warn', 'pooh', 'heffalump', 'woozl', 'steal', 'honey', 'heffalump', 'woozl', 'half', 'pooh', 'worri', 'huge', 'rain', 'storm', 'set', 'hundr', 'acr', 'wood', 'everybodi', 'swim', 'home', 'pooh', 'inadvert', 'save', 'piglet', 'waterfal', 'hero', 'parti', 'eeyor', 'barg', 'announc', 'found', 'new', 'home', 'owl', 'unfortun', 'hous', 'found', 'piglet', 'piglet', 'heroic', 'thing', 'turn', 'domicil', 'owl', 'two', 'hero', 'parti', 'storm', 'die', 'least', 'next', 'windsday'], ['imagin', 'disney', 'version', 'robin', 'hood', 'legend', 'fun', 'romanc', 'abound', 'swashbuckl', 'hero', 'sherwood', 'forest', 'valiant', 'sidekick', 'plot', 'one', 'dare', 'adventur', 'anoth', 'outwit', 'greedi', 'princ', 'partner', 'put', 'tax', 'squeez', 'poor'], ['magic', 'scepter', 'accident', 'transport', 'april', 'back', 'time', '17th', 'centuri', 'japan', 'boy', 'take', 'hot', 'pursuit', 'cowabungl', 'way', 'sewer', 'right', 'samurai', 'rama', 'must', 'battl', 'evil', 'lord', 'norinaga', 'reclaim', 'magic', 'scepter', 'bring', 'back', 'subway', 'new', 'york', 'citi'], ['gotham', 'citi', 'crime', 'boss', 'carl', 'grissom', 'jack', 'palanc', 'effect', 'run', 'town', 'new', 'crime', 'fighter', 'town', 'batman', 'michael', 'keaton', 'grissom', 'right', 'hand', 'man', 'jack', 'napier', 'jack', 'nicholson', 'brutal', 'man', 'entir', 'sane', '...', 'fall', 'two', 'grissom', 'napier', 'set', 'polic', 'napier', 'fall', 'appar', 'death', 'vat', 'chemic', 'howev', 'soon', 'reappear', 'joker', 'start', 'reign', 'terror', 'gotham', 'citi', 'meanwhil', 'report', 'vicki', 'vale', 'kim', 'basing', 'citi', 'articl', 'batman', 'soon', 'start', 'relationship', 'batman', 'everyday', 'persona', 'billionair', 'bruce', 'wayn'], ['sky', 'soviet', 'nicaraguan', 'cuban', 'troop', 'begin', 'land', 'footbal', 'field', 'colorado', 'high', 'school', 'second', 'paratroop', 'attack', 'school', 'sent', 'group', 'teenag', 'flee', 'mountain', 'arm', 'hunt', 'rifl', 'pistol', 'bow', 'arrow', 'teen', 'struggl', 'surviv', 'bitter', 'winter', 'soviet', 'kgb', 'patrol', 'hunt', 'eventu', 'troubl', 'aris', 'kill', 'group', 'soviet', 'soldier', 'patrol', 'highland', 'soon', 'wage', 'guerrilla', 'warfar', 'invad', 'soviet', 'troop', 'banner', 'wolverin'], ['martin', 'blank', 'freelanc', 'hitman', 'start', 'develop', 'conscienc', 'caus', 'muff', 'coupl', 'routin', 'assign', 'advic', 'secretari', 'psychiatrist', 'attend', '10th', 'year', 'high', 'school', 'reunion', 'gross', 'point', 'michigan', 'detroit', 'suburb', 'also', 'contract', 'kill', 'someon', 'hot', 'tail', 'coupl', 'enthusiast', 'feder', 'agent', 'anoth', 'assassin', 'want', 'kill', 'grocer', 'assassin', 'want', 'join', 'assassin', 'union'], ['elder', 'man', 'read', 'book', 'princess', 'bride', 'sick', 'thus', 'current', 'bedridden', 'adolesc', 'grandson', 'read', 'book', 'pass', 'within', 'famili', 'generat', 'grandson', 'sure', 'like', 'stori', 'romanc', 'core', 'prefer', 'someth', 'lot', 'action', 'kiss', 'grandson', 'powerless', 'stop', 'grandfath', 'whose', 'feel', 'want', 'hurt', 'stori', 'center', 'buttercup', 'former', 'farm', 'girl', 'chosen', 'princess', 'bride', 'princ', 'humperdinck', 'florian', 'buttercup', 'love', 'still', 'lament', 'death', 'one', 'true', 'love', 'westley', 'five', 'year', 'ago', 'westley', 'hire', 'hand', 'farm', 'stock', 'answer', 'wish', 'request', 'made', 'came', 'understand', 'way', 'say', 'love', 'westley', 'went', 'away', 'sea', 'kill', 'dread', 'pirat', 'robert', 'hors', 'ride', 'clear', 'mind', 'upcom', 'predica', 'marriag', 'buttercup', 'kidnap', 'band', 'bandit', 'vizzini', 'work', 'wit', 'two', 'associ', 'giant', 'name', 'fezzik', 'work', 'brawn', 'spaniard', 'name', 'inigo', 'montoya', 'train', 'entir', 'life', 'expert', 'swordsman', 'turn', 'chase', 'dread', 'pirat', 'robert', 'chase', 'princ', 'men', 'led', 'count', 'tyron', 'rugen', 'happen', 'collect', 'depend', 'part', 'buttercup', 'want', 'marri', 'princ', 'may', 'see', 'option', 'lesser', 'evil', 'part', 'motiv', 'individu', 'within', 'group', 'larger', 'question', 'grandson', 'think', 'stori', 'proceed', 'end', 'especi', 'see', 'justic', 'high', 'prioriti', 'action'], ['abandon', 'accid', 'babi', 'mowg', 'taken', 'rais', 'famili', 'wolv', 'boy', 'grow', 'older', 'wise', 'panther', 'bagheera', 'realiz', 'must', 'return', 'kind', 'nearbi', 'man', 'villag', 'baloo', 'bear', 'howev', 'think', 'differ', 'take', 'young', 'mowg', 'wing', 'teach', 'live', 'jungl', 'best', 'life', 'bagheera', 'realiz', 'mowg', 'danger', 'particular', 'shere', 'khan', 'tiger', 'hate', 'peopl', 'baloo', 'final', 'come', 'around', 'mowg', 'run', 'jungl', 'surviv', 'second', 'encount', 'kaa', 'snake', 'final', 'shere', 'khan', 'sight', 'pretti', 'girl', 'howev', 'get', 'mowg', 'nearbi', 'man', 'villag'], ['major', 'attitud', 'problem', 'histori', 'get', 'thing', 'done', 'told', 'interview', 'militari', 'prison', 'death', 'sentenc', 'long', 'term', 'danger', 'mission', 'parachut', 'behind', 'enemi', 'line', 'caus', 'havoc', 'german', 'general', 'rest', 'hous', 'eve', 'day'], ['bond', 'back', 'next', 'mission', 'take', 'fort', 'knox', 'auric', 'goldfing', 'henchman', 'plan', 'raid', 'fort', 'knox', 'obliter', 'world', 'economi', 'save', 'world', 'bond', 'need', 'becom', 'friend', 'goldfing', 'dodg', 'killer', 'hat', 'avoid', 'goldfing', 'person', 'pilot', 'sexi', 'pussi', 'galor', 'might', 'feel', 'bond', '007', 'help', 'chang', 'mind'], ['last', 'day', 'school', 'high', 'school', 'small', 'town', 'texa', '1976', 'upperclassmen', 'haze', 'incom', 'freshmen', 'everyon', 'tri', 'get', 'stone', 'drunk', 'laid', 'even', 'footbal', 'player', 'sign', 'pledg'], ['sean', 'thornton', 'return', 'america', 'reclaim', 'homestead', 'escap', 'past', 'sean', 'eye', 'caught', 'mari', 'kate', 'danah', 'beauti', 'poor', 'maiden', 'younger', 'sister', 'ill', 'temper', 'red', 'danah', 'riotous', 'relationship', 'form', 'sean', 'mari', 'kate', 'punctuat', 'pugnaci', 'attempt', 'keep', 'apart', 'form', 'main', 'plot', 'sean', 'past', 'dark', 'undercurr'], ['six', 'crimin', 'stranger', 'hire', 'crime', 'boss', 'joe', 'cabot', 'carri', 'diamond', 'robberi', 'right', 'outset', 'given', 'fals', 'name', 'intent', 'get', 'close', 'concentr', 'job', 'instead', 'complet', 'sure', 'robberi', 'go', 'success', 'polic', 'show', 'right', 'time', 'site', 'robberi', 'panic', 'spread', 'amongst', 'group', 'member', 'two', 'kill', 'subsequ', 'shootout', 'along', 'policemen', 'civilian', 'remain', 'peopl', 'assembl', 'premedit', 'rendezv', 'point', 'warehous', 'begin', 'suspect', 'one', 'undercov', 'cop'], ['scaramanga', 'hit', 'man', 'charg', 'million', 'dollar', 'per', 'job', 'becom', 'link', 'death', 'scientist', 'work', 'power', 'solar', 'cell', 'jame', 'bond', 'call', 'investig', 'track', 'scaramanga', 'realis', 'high', 'respect', 'killer', 'prove', 'advantag', 'final', 'showdown'], ['disney', 'anim', 'set', 'pictur', 'western', 'classic', 'music', 'leopold', 'stokowski', 'conduct', 'philadelphia', 'orchestra', 'sorcer', 'apprentic', 'featur', 'mickey', 'mous', 'aspir', 'magician', 'overstep', 'limit', 'rite', 'spring', 'tell', 'stori', 'evolut', 'singl', 'cell', 'anim', 'death', 'dinosaur', 'danc', 'hour', 'comic', 'ballet', 'perform', 'ostrich', 'hippo', 'eleph', 'allig', 'night', 'bald', 'mountain', 'ave', 'maria', 'set', 'forc', 'dark', 'light', 'devilish', 'revel', 'interrupt', 'come', 'new', 'day'], ['wwii', 'england', 'charli', 'carri', 'paul', 'rawlin', 'sent', 'live', 'eglantin', 'price', 'apprentic', 'witch', 'charli', 'blackmail', 'miss', 'price', 'keep', 'practic', 'secret', 'must', 'give', 'someth', 'take', 'bedknob', 'late', 'father', 'bed', 'place', 'famous', 'magic', 'travel', 'spell', 'paul', 'activ', 'first', 'journey', 'street', 'london', 'meet', 'emelius', 'brown', 'headmast', 'miss', 'price', 'witchcraft', 'train', 'correspond', 'school', 'miss', 'price', 'tell', 'plan', 'find', 'magic', 'word', 'spell', 'known', 'substitutiari', 'locomot', 'bring', 'inanim', 'object', 'life', 'spell', 'work', 'war', 'effort'], ['stoney', 'dave', 'find', 'caveman', 'link', 'trap', 'ice', 'thaw', 'show', 'around', 'town', 'although', 'link', 'slow', 'catch', 'basic', 'concept', '20th', 'centuri', 'life', 'troubl', 'impress', 'girl', 'help', 'stoney', 'dave', 'find', 'cool', 'search'], ['huge', 'advanc', 'scientif', 'technolog', 'enabl', 'mogul', 'creat', 'island', 'full', 'live', 'dinosaur', 'john', 'hammond', 'invit', 'four', 'individu', 'along', 'two', 'grandchildren', 'join', 'jurass', 'park', 'everyth', 'accord', 'plan', 'park', 'employe', 'attempt', 'steal', 'dinosaur', 'embryo', 'critic', 'secur', 'system', 'shut', 'becom', 'race', 'surviv', 'dinosaur', 'roam', 'freeli', 'island'], ['mikey', 'walsh', 'brandon', 'walsh', 'brother', 'whose', 'famili', 'prepar', 'move', 'develop', 'want', 'build', 'golf', 'cours', 'place', 'neighborhood', 'unless', 'enough', 'money', 'rais', 'stop', 'construct', 'golf', 'cours', 'quit', 'doubt', 'mikey', 'stumbl', 'upon', 'treasur', 'map', 'fame', 'one', 'eye', 'willi', 'hidden', 'fortun', 'mikey', 'brandon', 'friend', 'lawrenc', 'chunk', 'cohen', 'clark', 'mouth', 'devereaux', 'andrea', 'andi', 'carmichael', 'stefani', 'stef', 'steinbrenn', 'richard', 'data', 'wang', 'call', 'gooni', 'set', 'quest', 'find', 'treasur', 'hope', 'save', 'neighborhood', 'treasur', 'cavern', 'entranc', 'cavern', 'restaur', 'evil', 'thief', 'mama', 'fratelli', 'son', 'jake', 'fratelli', 'franci', 'fratelli', 'sever', 'disfigur', 'lotney', 'sloth', 'fratelli', 'sloth', 'befriend', 'gooni', 'decid', 'help'], ['luke', 'skywalk', 'battl', 'horribl', 'jabba', 'hut', 'cruel', 'darth', 'vader', 'save', 'comrad', 'rebel', 'allianc', 'triumph', 'galact', 'empir', 'han', 'solo', 'princess', 'leia', 'reaffirm', 'love', 'team', 'chewbacca', 'lando', 'calrissian', 'ewok', 'android', '3po', 'aid', 'disrupt', 'dark', 'side', 'defeat', 'evil', 'emperor'], ['tell', 'stori', 'day', 'invas', 'normandi', 'wwii', 'dozen', 'charact', 'seen', 'briefli', 'togeth', 'weav', 'stori', 'five', 'separ', 'invas', 'point', 'made', 'oper'], ['charm', 'film', 'base', 'popular', 'frank', 'baum', 'stori', 'dorothi', 'dog', 'toto', 'caught', 'tornado', 'path', 'somehow', 'end', 'land', 'meet', 'memor', 'friend', 'foe', 'journey', 'meet', 'wizard', 'everyon', 'say', 'help', 'return', 'home', 'possibl', 'grant', 'new', 'friend', 'goal', 'brain', 'heart', 'courag'], ['maximus', 'power', 'roman', 'general', 'love', 'peopl', 'age', 'emperor', 'marcus', 'aurelius', 'death', 'emperor', 'choos', 'maximus', 'heir', 'son', 'commodus', 'power', 'struggl', 'leav', 'maximus', 'famili', 'condemn', 'death', 'power', 'general', 'unabl', 'save', 'famili', 'loss', 'allow', 'get', 'captur', 'put', 'gladiat', 'game', 'die', 'desir', 'fuel', 'chanc', 'rise', 'top', 'abl', 'look', 'eye', 'man', 'feel', 'reveng'], ['two', 'young', 'hippi', 'biker', 'wyatt', 'billi', 'sell', 'dope', 'southern', 'california', 'stash', 'money', 'away', 'gas', 'tank', 'set', 'trip', 'across', 'america', 'person', 'odyssey', 'look', 'way', 'lead', 'live', 'journey', 'encount', 'bigotri', 'hatr', 'small', 'town', 'communiti', 'despis', 'fear', 'non', 'conform', 'howev', 'wyatt', 'billi', 'also', 'discov', 'peopl', 'attempt', 'altern', 'lifestyl', 'resist', 'narrow', 'minded', 'alway', 'question', 'mark', 'futur', 'surviv', 'drop', 'group', 'gentl', 'hippi', 'communiti', 'thank', 'god', 'place', 'stand', 'live', 'unreal', 'dream', 'rancher', 'encount', 'mexican', 'wife', 'hard', 'push', 'make', 'end', 'meet', 'even', 'lsd', 'turn', 'sour', 'trip', 'bad', 'one', 'death', 'come', 'seem', 'freedom', 'arriv', 'diner', 'small', 'town', 'insult', 'local', 'redneck', 'weirdo', 'degener', 'arrest', 'minor', 'pretext', 'local', 'sheriff', 'thrown', 'jail', 'meet', 'georg', 'hanson', 'liber', 'alcohol', 'lawyer', 'get', 'decid', 'join', 'trip', 'new', 'orlean', 'time', 'mardi', 'gras'], ['begin', 'twentieth', 'centuri', 'mrs', 'edwin', 'muir', 'luci', 'widow', 'one', 'year', 'decid', 'move', 'control', 'law', 'home', 'london', 'english', 'seasid', 'adolesc', 'daughter', 'anna', 'long', 'devot', 'maid', 'martha', 'despit', 'rental', 'agent', 'tri', 'dissuad', 'luci', 'decid', 'rent', 'gull', 'cottag', 'whitecliff', 'sea', 'learn', 'first', 'hand', 'make', 'decis', 'rental', 'agent', 'hesit', 'cottag', 'haunt', 'suppos', 'deceas', 'former', 'owner', 'seaman', 'captain', 'daniel', 'gregg', 'move', 'meet', 'spirit', 'captain', 'gregg', 'face', 'face', 'refus', 'scare', 'away', 'presenc', 'two', 'come', 'understand', 'includ', 'make', 'presenc', 'known', 'anna', 'time', 'progress', 'two', 'develop', 'friendship', 'bond', 'despit', 'statement', 'need', 'live', 'life', 'includ', 'find', 'anoth', 'husband', 'daniel', 'seem', 'approv', 'men', 'enter', 'life', 'includ', 'serious', 'children', 'author', 'mile', 'fairley', 'feel', 'daniel', 'eventu', 'decid', 'part', 'life', 'benefit', 'hindranc', 'carri', 'live', 'regardless', 'perhap', 'abl', 'carri', 'initi', 'goal', 'realiz', 'vision', 'gull', 'cottag', 'leav'], ['joan', 'wilder', 'mousi', 'romanc', 'novelist', 'receiv', 'treasur', 'map', 'mail', 'recent', 'murder', 'brother', 'law', 'meanwhil', 'sister', 'elain', 'kidnap', 'colombia', 'two', 'crimin', 'respons', 'demand', 'travel', 'colombia', 'exchang', 'map', 'sister', 'joan', 'quick', 'becom', 'lost', 'jungl', 'waylay', 'zolo', 'vicious', 'corrupt', 'colombian', 'cop', 'stop', 'noth', 'obtain', 'map', 'meet', 'irrever', 'soldier', 'fortun', 'name', 'jack', 'colton', 'agre', 'bring', 'back', 'civil', 'togeth', 'embark', 'upon', 'adventur', 'could', 'straight', 'joan', 'novel'], ['last', 'member', 'die', 'nativ', 'american', 'tribe', 'mohican', 'unca', 'father', 'chingachgook', 'adopt', 'half', 'white', 'brother', 'hawkey', 'live', 'peac', 'alongsid', 'british', 'colonist', 'daughter', 'british', 'colonel', 'kidnap', 'traitor', 'scout', 'hawkey', 'unca', 'must', 'rescu', 'crossfir', 'gruesom', 'militari', 'conflict', 'want', 'part', 'french', 'indian', 'war'], ['myle', 'divorc', 'want', 'love', 'life', 'film', 'career', 'decid', 'date', 'find', 'true', 'love', 'front', 'camera', 'make', 'first', 'featur', 'patient', 'agent', 'richard', 'find', '000', 'investor', 'shadowi', 'eli', 'myle', 'start', 'search', 'sometim', 'tell', 'date', 'film', 'sometim', 'eli', 'want', 'sex', 'titil', 'myle', 'want', 'real', 'myle', 'regular', 'talk', 'old', 'film', 'teacher', 'robert', 'mckee', 'wonder', 'love', 'possibl', 'modern', 'life', 'half', 'way', 'date', 'myle', 'meet', 'elisabeth', 'everyth', 'desir', 'like', 'finish', 'date', 'satisfi', 'eli', 'complet', 'film', 'without', 'lose', 'elisabeth'], ['upon', 'releas', 'mental', 'hospit', 'follow', 'nervous', 'breakdown', 'directionless', 'anthoni', 'join', 'friend', 'dignan', 'seem', 'far', 'less', 'sane', 'former', 'dignan', 'hatch', 'hare', 'brain', 'scheme', 'yet', 'unspecifi', 'crime', 'spree', 'somehow', 'involv', 'former', 'boss', 'suppos', 'legendari', 'henri', 'help', 'pathet', 'neighbor', 'pal', 'bob', 'anthoni', 'dignan', 'pull', 'job', 'hit', 'road', 'anthoni', 'find', 'love', 'motel', 'maid', 'inez', 'boy', 'final', 'hook', 'henri', 'ensu', 'escapad', 'turn', 'far', 'anyon', 'expect'], ['economi', 'rut', 'presid', 'approv', 'rate', 'need', 'good', 'war', 'russian', 'interest', 'hey', 'big', 'polit', 'countri', 'north', 'niagara', 'fall', 'sheriff', 'bud', 'boomer', 'take', 'bit', 'serious', 'though'], ['henri', 'frankenstein', 'doctor', 'tri', 'discov', 'way', 'make', 'dead', 'walk', 'succeed', 'creat', 'monster', 'deal', 'live'], ['beauti', 'princess', 'born', 'royalti', 'everyon', 'gather', 'exchang', 'gift', 'everyth', 'perfect', 'fine', 'unwant', 'guest', 'appear', 'magnifi', 'magnific', 'curs', 'spell', 'young', 'princess', 'tell', 'fall', 'asleep', 'sunset', 'even', '16th', 'birthday', 'way', 'wake', 'tear', 'true', 'love', 'final', 'day', 'come', 'left', 'sleep', 'forev'], ['world', 'mutant', 'human', 'fear', 'mari', 'better', 'known', 'rogu', 'run', 'away', 'home', 'hitch', 'ride', 'anoth', 'mutant', 'known', 'logan', 'wolverin', 'charl', 'xavier', 'own', 'school', 'young', 'mutant', 'send', 'storm', 'cyclop', 'bring', 'back', 'late', 'magneto', 'believ', 'war', 'approach', 'evil', 'plan', 'mind', 'need', 'young', 'rogu', 'help'], ['wisbourg', 'germani', 'base', 'estat', 'agent', 'knock', 'dispatch', 'associ', 'hutter', 'count', 'orlok', 'castl', 'transylvania', 'count', 'want', 'purchas', 'isol', 'hous', 'wisbourg', 'plan', 'sell', 'one', 'across', 'way', 'hutter', 'home', 'hutter', 'leav', 'innoc', 'wife', 'ellen', 'friend', 'away', 'hutter', 'trek', 'unusu', 'one', 'mani', 'local', 'want', 'take', 'near', 'castl', 'strang', 'event', 'occur', 'castl', 'hutter', 'manag', 'sell', 'count', 'hous', 'also', 'notic', 'feel', 'unusu', 'occurr', 'primarili', 'feel', 'like', 'dark', 'shadow', 'hang', 'even', 'daytim', 'count', 'unusu', 'asleep', 'hutter', 'eventu', 'see', 'count', 'sleep', 'chamber', 'crypt', 'base', 'book', 'recent', 'read', 'believ', 'count', 'realli', 'vampir', 'nosferatu', 'hutter', 'trap', 'castl', 'count', 'hide', 'shipment', 'coffin', 'make', 'way', 'wisbourg', 'caus', 'death', 'along', 'way', 'attribut', 'plagu', 'hutter', 'tri', 'rush', 'home', 'save', 'town', 'import', 'save', 'ellen', 'nosferatu', 'immin', 'arriv', 'wisbourg', 'ellen', 'feel', 'impend', 'dark', 'nosferatu', 'get', 'closer', 'learn', 'sinless', 'woman', 'sacrific', 'kill', 'vampir', 'hutter', 'abl', 'save', 'ellen', 'either', 'nosferatu', 'self', 'sacrific'], ['destruct', 'planet', 'krypton', 'scientist', 'jor', 'send', 'infant', 'son', 'kal', 'spaceship', 'earth', 'rais', 'kind', 'farmer', 'jonathan', 'martha', 'kent', 'young', 'clark', 'discov', 'sourc', 'superhuman', 'power', 'move', 'metropoli', 'fight', 'evil', 'superman', 'battl', 'villain', 'lex', 'luthor', 'novic', 'report', 'clark', 'kent', 'attempt', 'woo', 'worker', 'loi', 'lane']]

def computeTF(freqDict_list):
    TF_scores=[]
    print("Calculating TF")
    for tempDict in freqDict_list:
        id=tempDict['term_id']
        name=tempDict['movie_id']
        for k in tempDict['freq_dict']:
            temp={'term_id':id, 'movie_id':name,'TF_score':tempDict['freq_dict'][k]/tempDict['doc_length'],'key':k}
            TF_scores.append(temp)
    return TF_scores

def computeIDF(freqDict_list):
    IDF_scores=[]
    print("Calculating IDF")
    counter=-1
    for dict in freqDict_list:
       counter+=1
       for k in dict['freq_dict'].keys():
            count=sum([k in tempDict['freq_dict'] for tempDict in freqDict_list])
            temp= {'term_id': counter, 'IDF_score':math.log(dict['doc_length'])/count,'key':k}
            IDF_scores.append(temp)
    return IDF_scores

def computeTFIDF(TF_scores, IDF_scores):
    TF_IDF_scores=[]
    print("Calculating TF-IDF")
    
    for j in IDF_scores:
        for i in TF_scores:
            if j['key']==i['key'] and j['term_id']==i['term_id']:
                temp={'term_id':j['term_id'], 'movie_id':i['movie_id'], 'TFIDF_score': j['IDF_score']*i['TF_score'], 'key':i['key']}
                TF_IDF_scores.append(temp)
    return TF_IDF_scores

freqDict=create_freq_dict(user_movies, processedText)
TFIDF=computeTFIDF(computeTF(freqDict),computeIDF(freqDict))

Creating frequency dictionary
{'term_id': 0, 'movie_id': '0114709', 'freq_dict': {'hunter': 1, 'prey': 1, 'neil': 3, 'profession': 1, 'crimin': 2, 'crew': 1, 'hunt': 2, 'score': 2, 'big': 2, 'money': 1, 'target': 1, 'bank': 1, 'vault': 1, 'armor': 1, 'car': 1, 'turn': 1, 'vincent': 2, 'hanna': 2, 'team': 1, 'cop': 1, 'robberi': 1, 'homicid': 1, 'polic': 1, 'divis': 1, 'botch': 1, 'job': 1, 'put': 2, 'onto': 1, 'trail': 1, 'regroup': 1, 'tri': 1, 'togeth': 1, 'one': 1, 'last': 1, 'retir': 1, 'similar': 1, 'mani': 1, 'way': 1, 'includ': 1, 'troubl': 1, 'person': 1, 'live': 1, 'crucial': 1, 'moment': 1, 'life': 2, 'disobey': 1, 'dictum': 1, 'taught': 1, 'long': 1, 'ago': 1, 'mentor': 1, '--': 2, 'never': 1, 'anyth': 1, 'walk': 1, 'thirti': 1, 'second': 1, 'flat': 1, 'spot': 1, 'heat': 1, 'come': 1, 'around': 1, 'corner': 1, 'fall': 1, 'love': 1, 'thus': 1, 'stage': 1, 'set': 1, 'suspens': 1, 'end': 1, '....': 1}, 'doc_length': 82}
{'term_id': 1, 'movie_id': '0113228', 'freq_dict': {'inven

Calculating TF-IDF


In [ ]:
def content_based(user_id, hybrid=True):

  movie_similarities=[]
  temp_user_movies=[]
  temp_movies=[]
  user_movies=[]
  #Get movies of selected user
  for k in range(0,len(ds_ratings)-1):
    if str(ds_ratings['userId'][k])==str(user_id):
      temp_user_movies.append(ds_ratings['movieId'][k])
  #Get imbdIds of user movies
  for k in range(0,len(ds_links)-1):
    if str(ds_links['movieId'][k]) in temp_user_movies:
      user_movies.append(str(ds_links['imdbId'][k]))
  
  movies=[]
  
  #If we are using content-based as part of the hybrid filter, it uses the results of the collaborative filtering
  #Otherwise it uses all movies
  if(hybrid):
    temp_movies=[str(x[0]) for x in filter(user_id)]
    for k in range(0,len(ds_links)):
      if str(ds_links['movieId'][k]) in temp_movies:
        movies.append(str(ds_links['imdbId'][k]))
  else:
    movies=[str(movie_id) for movie_id in list_movies_imdbid if movie_id not in user_movies]
  #Preprocess user movies
  preprocessed_texts=[preprocess_text(text[0]) for text in readPlots(user_movies)]

  for i in range(0,len(movies)-1):
    MaxSim=0
    user_movies.append(movies[i])
    preprocessed_texts.append(preprocess_text(readPlots([movies[i]])[0][0]))
    freqDict=create_freq_dict(user_movies, preprocessed_texts)
    user_movies.pop(-1)
    preprocessed_texts.pop(-1)
    TFIDF=computeTFIDF(computeTF(freqDict),computeIDF(freqDict))

    TFIDF_highest_temp=[[y["key"],y["TFIDF_score"]] for y in TFIDF if y["movie_id"]==movies[i]]
    TFIDF_highest=[y[1] for y in TFIDF_highest_temp]
    #TFIDF_highest=list(map(lambda y : y["TFIDF_score"] if y["movie_id"]==movies[i] else 0, TFIDF))
    
    for j in range(0,9):
      
      TFIDF_user_temp={x["key"]:x["TFIDF_score"] for x in TFIDF if x["movie_id"]==user_movies[j]}
      TFIDF_user=[]
      for x in TFIDF_highest_temp:
        if x[0] in TFIDF_user_temp.keys():
          TFIDF_user.append(TFIDF_user_temp[x[0]])
        else:
          TFIDF_user.append(0)

      #TFIDF_user=list(map(lambda y : y["TFIDF_score"] if y["movie_id"]==user_movies[i] else 0, TFIDF))
      
      similarity=cosine_similarity(TFIDF_highest,TFIDF_user)
      if(similarity>MaxSim):
        MaxSim=similarity
    movie_similarities.append([movies[i],MaxSim])
  return sorted(movie_similarities, key=lambda item: item[1], reverse=True)

print(content_based(1))





Fuzzy Expert

In [ ]:
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import matplotlib.pyplot as plt

In [ ]:
inp1 = ctrl.Antecedent(np.arange(0, 5, 1), 'average_rating')
inp2 = ctrl.Antecedent(np.arange(0, 350, 1), 'total_ratings')
inp3 = ctrl.Antecedent(np.arange(0, 1, 1), 'similarity')
out = ctrl.Consequent(np.arange(0, 1, 1), 'importance')

out['low'] = fuzz.trimf(out.universe, [0, 0, .2])
out['medium'] = fuzz.trimf(out.universe, [0, .6, 25])
out['high'] = fuzz.trimf(out.universe, [.6, 1, 25])